# QALD Data Question and Query Translation
- Method 0: Answer the QALD-9 questions directly. Instruct ChatGPT to answer the QALD-9 questions directly without querying DBpedia
- Method 1: Translate QALD-9 questions directly. Instruct ChatGPT to translate QALD-9 questions directly, and then query DBpedia using the translated queries.
- Method 2: 1-shot learning from a pair of train question and query. Using the embeddings of the test and train questions to find the most similar train question to the test question. Prompt ChatGPT with the pair of matched train question and query. Instruct ChatGPT to translate a test question to a SPARQL query over DBpedia.
- Method 3: 1-shot learning from a pair of train question and query, and the chain-of-thought of the train query. As in Method 2, include the chain-of-thought of the train query in the prompt, in addition to the pair of matched question and query

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json, re, os, nltk

## Pre-processing QALD Train Data

In [2]:
with open('../data/QALD/9/data/qald-9-train-multilingual.json', 'r') as file:
    train_json = json.load(file)

In [3]:
train_all = pd.DataFrame(train_json['questions'])
train_all

,id,answertype,aggregation,onlydbo,hybrid,question,query,answers
0,1,resource,False,True,False,"[{'language': 'de', 'string': 'Liste alle Bret...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
1,2,resource,False,True,False,"[{'language': 'de', 'string': 'Wer entwickelt ...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
2,3,resource,False,False,False,"[{'language': 'de', 'string': 'Welche Menschen...",{'sparql': 'PREFIX yago: <http://dbpedia.org/c...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
3,4,resource,False,True,False,"[{'language': 'de', 'string': 'Im welche US Zu...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
4,5,resource,False,True,False,"[{'language': 'de', 'string': 'Wer ist der Bür...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
...,...,...,...,...,...,...,...,...
403,409,resource,False,True,False,"[{'language': 'de', 'string': 'Was war das ers...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
404,410,string,False,True,False,"[{'language': 'de', 'string': 'Was war der ers...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['string']}, 'results': {'b..."
405,411,resource,False,True,False,"[{'language': 'de', 'string': 'Was war der let...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
406,412,resource,False,True,False,"[{'language': 'de', 'string': 'Gib mir alle Sc...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."


In [4]:
train_all.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question',
       'query', 'answers'],
      dtype='object')

In [5]:
train_all.question[39]

[{'language': 'de',
  'string': 'Zeig mir alle Schwimmer, die in Moskau geboren wurden.',
  'keywords': 'Schwimmer, geboren in, Moskau'},
 {'language': 'ru',
  'string': 'Дайте все пловцы что мы Родился в Москва. ',
  'keywords': 'пловец ,  Родился ,  Москва '},
 {'language': 'pt',
  'string': 'Liste todos os nadadores nascido em Moscou.',
  'keywords': 'nadador ,  nascermos ,  Moscou '},
 {'language': 'en',
  'string': 'Give all swimmers that were born in Moscow.',
  'keywords': 'swimmer, born, Moscow'},
 {'language': 'hi_IN',
  'string': ' मॉस्को में पैदा हुए सभी तैराकों के नाम दें।',
  'keywords': 'तैराक, जन्म, मास्को'},
 {'language': 'fa',
  'string': 'تمام شناگرهایی که در مسکو متولد شدند را لیست کن.',
  'keywords': 'شناگرهایی، مسکو، متولد'},
 {'language': 'it',
  'string': 'Dammi tutti i nuotatori che sono nati a Mosca.',
  'keywords': 'nuotatori, nati, Mosca'},
 {'language': 'fr',
  'string': 'Donne-moi tous les nageurs nés à Moscou.',
  'keywords': 'nageurs, nés, Moscou'},
 {'la

### Flatten the QALD train data

In [50]:
questions = []
keywords = []
for idx, row in train_all.iterrows():
    question = row['question']
    for q in question:
        try:
            if q['language'] == 'en':
                questions.append(q['string'])
                keywords.append(q['keywords'])
                break
        except:
            print(q)

In [51]:
len(questions), len(keywords)

(408, 408)

In [52]:
queries = train_all['query'].apply(lambda r: r['sparql'])

In [53]:
len(queries)

408

In [55]:
heads = train_all['answers'].apply(lambda r: r[0]).apply(lambda p: p['head']) 

In [56]:
results = train_all['answers'].apply(lambda r: r[0]).apply(lambda p: p['results'])

In [57]:
train_all['question_text'] = questions

In [58]:
train_all['sparql_query'] = queries

In [59]:
train_all['question_keywords'] = keywords

In [60]:
train_all['answer_head'] = heads

In [61]:
train_all['answer_results'] = results

In [62]:
#train_all[['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text', 
#           'question_keywords', 'sparql_query', 'answer_head', 'answer_results', 'question', 
#           'query', 'answers']].to_csv('../data/QALD/9/data/qald-9-train.csv', index=None)

## Pre-processing QALD Test Data

In [63]:
with open('../data/QALD/9/data/qald-9-test-multilingual.json', 'r') as file:
    test_json = json.load(file)

In [65]:
test_all = pd.DataFrame(test_json['questions'])
test_all

,id,answertype,aggregation,onlydbo,hybrid,question,query,answers
0,99,resource,False,True,False,"[{'language': 'de', 'string': 'In welcher Zeit...",{'sparql': 'PREFIX res: <http://dbpedia.org/re...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
1,98,resource,False,False,False,"[{'language': 'de', 'string': 'Wer hat Caesar ...",{'sparql': 'PREFIX dct: <http://purl.org/dc/te...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
2,86,resource,False,True,False,"[{'language': 'de', 'string': 'Was ist das höc...",{'sparql': 'PREFIX rdfs: <http://www.w3.org/20...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
3,84,resource,False,False,False,"[{'language': 'de', 'string': 'Welche Amerikan...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
4,81,resource,False,False,False,"[{'language': 'de', 'string': 'Von welchem US-...",{'sparql': 'SELECT DISTINCT ?uri WHERE { ?uri ...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
...,...,...,...,...,...,...,...,...
145,14,resource,False,True,False,"[{'language': 'de', 'string': 'Zeig mir die Wa...",{'sparql': 'SELECT DISTINCT ?uri WHERE { ?uri ...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
146,87,resource,False,True,False,"[{'language': 'de', 'string': 'Welches Buch ha...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
147,148,resource,False,False,False,"[{'language': 'de', 'string': 'Was ist der grö...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
148,43,resource,False,False,False,"[{'language': 'de', 'string': 'Gib mir alle We...",{'sparql': 'PREFIX foaf: <http://xmlns.com/foa...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."


In [66]:
test_all.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question',
       'query', 'answers'],
      dtype='object')

In [67]:
test_all.question[0]

[{'language': 'de',
  'string': 'In welcher Zeitzone liegt Salt Lake City?',
  'keywords': 'Salt Lake City,  Zeitzone'},
 {'language': 'ru',
  'string': 'Какие является  время зона из Поваренная соль Озеро Город? ',
  'keywords': 'Поваренная соль Озеро город ,   время зона '},
 {'language': 'pt',
  'string': 'Qual é o fuso horário da Salt Lake City?',
  'keywords': 'Cidade de Salt Lake, fuso horário'},
 {'language': 'en',
  'string': 'What is the time zone of Salt Lake City?',
  'keywords': 'Salt Lake City,  time zone'},
 {'language': 'hi_IN',
  'string': 'साल्ट लेक सिटी का समय क्षेत्र क्या है?',
  'keywords': 'साल्ट लेक सिटी,  समय क्षेत्र'},
 {'language': 'fa',
  'string': 'منطقه زمانی سالت لیک سیتی چیست؟',
  'keywords': 'سالت\u200cلیک\u200cسیتی,  منطقه زمانی'},
 {'language': 'pt_BR',
  'string': 'Qual o fuso horário da cidade de Salt Lake?',
  'keywords': 'fuso horário, cidade de Salt Lake'},
 {'language': 'it',
  'string': 'In quale fuso orario si trova Salt Lake City?',
  'keywords

### Flatten the QALD test data

In [68]:
questions = []
keywords = []
for idx, row in test_all.iterrows():
    question = row['question']
    for q in question:
        try:
            if q['language'] == 'en':
                questions.append(q['string'])
                keywords.append(q['keywords'])
                break
        except:
            print(q)

In [69]:
len(questions), len(keywords)

(150, 150)

In [70]:
queries = test_all['query'].apply(lambda r: r['sparql'])

In [71]:
len(queries)

150

In [72]:
heads = test_all['answers'].apply(lambda r: r[0]).apply(lambda p: p['head']) 

In [73]:
results = test_all['answers'].apply(lambda r: r[0]).apply(lambda p: p['results'])

In [74]:
test_all['question_text'] = questions

In [75]:
test_all['sparql_query'] = queries

In [76]:
test_all['question_keywords'] = keywords

In [77]:
test_all['answer_head'] = heads

In [78]:
test_all['answer_results'] = results

In [79]:
#test_all[['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text', 
#           'question_keywords', 'sparql_query', 'answer_head', 'answer_results', 'question', 
#           'query', 'answers']].to_csv('../data/QALD/9/data/qald-9-test.csv', index=None)

## Load the train and test data

In [80]:
train = pd.read_csv('../data/QALD/9/data/qald-9-train.csv')
train.head()

,id,answertype,aggregation,onlydbo,hybrid,question_text,question_keywords,sparql_query,answer_head,answer_results,question,query,answers
0,1,resource,False,True,False,List all boardgames by GMT.,"boardgame, GMT",PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Liste alle Bret...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
1,2,resource,False,True,False,Who developed Skype?,"develop, Skype",PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Wer entwickelt ...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
2,3,resource,False,False,False,Which people were born in Heraklion?,"people, born, heraklion",PREFIX yago: <http://dbpedia.org/class/yago/> ...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Welche Menschen...",{'sparql': 'PREFIX yago: <http://dbpedia.org/c...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
3,4,resource,False,True,False,In which U.S. state is Area 51 located?,"Area 51, located, U.S. state",PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Im welche US Zu...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
4,5,resource,False,True,False,Who is the mayor of New York City?,"New York City, mayor",PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Wer ist der Bür...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."


In [81]:
test = pd.read_csv('../data/QALD/9/data/qald-9-test.csv')
test.head()

,id,answertype,aggregation,onlydbo,hybrid,question_text,question_keywords,sparql_query,answer_head,answer_results,question,query,answers
0,99,resource,False,True,False,What is the time zone of Salt Lake City?,"Salt Lake City, time zone",PREFIX res: <http://dbpedia.org/resource/> PRE...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'In welcher Zeit...",{'sparql': 'PREFIX res: <http://dbpedia.org/re...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
1,98,resource,False,False,False,Who killed Caesar?,"who , killed, Caesar",PREFIX dct: <http://purl.org/dc/terms/> PREFIX...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Wer hat Caesar ...",{'sparql': 'PREFIX dct: <http://purl.org/dc/te...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
2,86,resource,False,True,False,What is the highest mountain in Germany?,"highest, mountain, germany",PREFIX rdfs: <http://www.w3.org/2000/01/rdf-sc...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Was ist das höc...",{'sparql': 'PREFIX rdfs: <http://www.w3.org/20...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
3,84,resource,False,False,False,Which American presidents were in office durin...,"American presidents, office, Vietnam War",PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Welche Amerikan...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
4,81,resource,False,False,False,Butch Otter is the governor of which U.S. state?,"U.S. state, governor, Butch Otter",SELECT DISTINCT ?uri WHERE { ?uri a <http://db...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Von welchem US-...",{'sparql': 'SELECT DISTINCT ?uri WHERE { ?uri ...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."


In [82]:
with open('../data/QALD/9/data/qald-9-test-multilingual.json', 'r') as file:
    test_json = json.load(file)

In [83]:
test_json_df = pd.DataFrame(test_json['questions'])

## Ask Test Questions through GPT-3

In [82]:
import os
openai_key_path = "OPENAI_KEY_PATH"
with open(openai_key_path, 'r') as f:
    openai_key = f.readline()
# My OpenAI Key
os.environ['OPENAI_API_KEY'] = openai_key

In [ ]:
import os
import openai

In [ ]:
from tqdm import tqdm

In [ ]:
gpt_answers = []
for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    
    question_text = row['question_text']
    keywords = row['question_keywords']
    
    answertype = row['answertype']
    answertype_text = ""
    if answertype == 'resource':
        answertype_text = 'resource in DBpedia-2016-04'
    else:
        answertype_text = answertype
    
    answeragg = row['aggregation']
    answeragg_text = "The answers do not need aggregation"
    if answeragg:
        answeragg_text = "The answers need aggregration"
        
    prompt_template = "Use DBpedia-2016-04 knowledge base. \
        Answer the question. No comments. List answers only. \n \
        The keywords in the question are \"{}\". \n\
        Output the answers as {}. \n\
        {}. \n\
        QUESTION: {} \n \
        ANSWER: \n"
    
    prompt = prompt_template.format(keywords, answertype_text, answeragg_text, 
                                    question_text)
    
    
    response = openai.Completion.create(
        #model="text-curie-001",
        #model="text-davinci-003",
        prompt= prompt,
        #prompt=question_text,
        temperature=0.1,
        max_tokens=3500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    gpt_answers.append(response)

In [ ]:
len(gpt_answers)

In [ ]:
import pickle

In [ ]:
#with open('../data/QALD/9/data/qald-test-qpt3-answers-question-keywords.pk', 'wb') as f:
    #pickle.dump(gpt_answers, f)

### Evaluate the GPT_Answers on Test 

In [ ]:
with open('../data/QALD/9/data/qald-test-qpt3-answers-question-keywords.pk', 'rb') as f:
    gpt_answers = pickle.load(f)

In [ ]:
len(gpt_answers)

In [241]:
# extract the answer terms from gpt_answer_text
gpt_answer_terms = []
for idx, row in test.iterrows():
    answers_text = row['gpt_answers_text']
    terms = answers_text.replace('http://dbpedia.org/resource/', '').replace('dbo:', '').strip().lower().split('\n')
    #terms = answers_text.strip().lower().split('\n')
    gpt_answer_terms.append([t.strip() for t in terms])

In [242]:
len(gpt_answer_terms)

150

In [243]:
import urllib.parse

gpt_answer_terms_parsed = []
for terms in gpt_answer_terms:
    terms_parsed = []
    for term in terms:
        parsed_string = urllib.parse.unquote(term)
        terms_parsed.append(parsed_string)
    gpt_answer_terms_parsed.append(terms_parsed)

In [215]:
gpt_answer_terms_parsed

[['mountain_time_zone'],
 ['marcus_junius_brutus'],
 ['zugspitze'],
 ['<lyndon_b._johnson>', '<richard_nixon>', '<gerald_ford>', '<jimmy_carter>'],
 ['idaho'],
 ['8'],
 ['karen_carpenter',
  'kylie_minogue',
  'kurt_cobain',
  'kerry_katona',
  'kelis',
  'kylie_bax'],
 ['writer'],
 ['true'],
 ['50,000'],
 ['star_alliance'],
 ['united_kingdom',
  'spain',
  'belgium',
  'denmark',
  'luxembourg',
  'monaco',
  'netherlands',
  'norway',
  'sweden'],
 ['austria',
  'belgium',
  'croatia',
  'czech_republic',
  'france',
  'germany',
  'greece',
  'hungary',
  'italy',
  'poland',
  'romania',
  'serbia',
  'slovenia',
  'spain',
  'switzerland',
  'ukraine'],
 ['los_angeles_international_airport',
  'san_francisco_international_airport',
  'san_diego_international_airport',
  'sacramento_international_airport',
  'oakland_international_airport',
  'fresno_yosemite_international_airport',
  'palm_springs_international_airport',
  'ontario_international_airport',
  'john_wayne_airport',
 

In [244]:
answer_terms = []
count = 0
for idx, row in test_json_df.iterrows():
    try:
        bindings = row['answers'][0]['results']['bindings']

        answer_list = []
        for item in bindings:
            for k in item:
                answer_list.append(item[k]['value'])

        terms = []
        for ans in answer_list:
            terms.append(ans.replace('http://dbpedia.org/resource/', '').replace('dbo:', '').strip().lower())
        #if terms not in answer_terms:
        answer_terms.append(terms)
              
    except:
        answer_terms.append([str(row['answers'][0]['boolean']).lower()])
        count += 1
        #print(row['answers'])

In [143]:
len(answer_terms)

150

In [245]:
predicted = 0
gold = 0
predicted_correct = 0
some_matched = {}
for idx, pred_terms in enumerate(gpt_answer_terms_parsed):
    gold_terms = answer_terms[idx]
    
    predicted +=  len(pred_terms)
    gold += len(gold_terms)
    
    predicted_correct_idx = 0
    for pterm in pred_terms:
        if len(pterm) > 0: # skip an empty string
            for gterm in gold_terms:
                #if pterm ==  gterm:
                if (pterm in gterm) or (gterm in pterm):
                    predicted_correct_idx += 1
                    predicted_correct += 1
                    break # skip correct prediction, don't double count anymore
                
    some_matched[idx] = predicted_correct_idx

In [246]:
precision = predicted_correct / predicted
precision

0.4945332211942809

In [247]:
recall = predicted_correct/gold
recall

0.1279930343926861

In [248]:
f1 = 2 / (1/precision + 1/recall)
f1

0.20335466021096316

## Use ChatGPT to answer and translate the test questions 
- Use ChatGPT to answer the questions directly
- Use ChatGPT to translate user questions to queries
- Use ChatGPT to translate user questions to queries by few-shot learning
- Use ChatGPT to translate user questions to queries by few-shot learning and chain of thought

### User ChatGPT to answer the question directly on DBpedia-03132023

In [85]:
test.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers'],
      dtype='object')

In [ ]:
msg = """
Answer the question. Output the answer only. No comments in the output. 
The keywords in the question are "{}". 
Output the answers as "{}". 
If the question cannot be answered using the DBpedia-2016-04 knowledge base, 
output "None".

QUESTION: {}

ANSWER:
"""

In [23]:
from tqdm import tqdm

In [166]:
chatgpt_answers = []
count = 0
for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    #if count > 3:
    #    break
     
    count += 1
    question_text = row['question_text']
    keywords = row['question_keywords']
    
    answertype = row['answertype']
    answertype_text = ""
    if answertype == 'resource':
        answertype_text = 'DBpedia Resource URI(s)'
    else:
        answertype_text = answertype
    
    answeragg = row['aggregation']
    answeragg_text = "The answers do not need aggregation"
    if answeragg:
        answeragg_text = "The answers need aggregration"
        
    msg = """
        Answer the question. No comments. List answers only.  
        The keywords in the question are {}. 
        Output the answers as {}. 

        QUESTION: {}

        ANSWER: 
    """
    
    msg = msg.format(keywords, answertype_text, question_text)
    
    #print(msg)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant using \
               DBpedia to answer questions."},
            {"role": "user", "content": msg}
        ]
    )

    chatgpt_answers.append(response['choices'][0]['message']['content'])

100%|██████████| 150/150 [14:19<00:00,  5.73s/it]


In [ ]:
test['chatgpt_answers_text_DBpedia_2023_03'] = chatgpt_answers

#### Evaluate the results of using ChatGPT to answer the questions directly on DBpedia-03132023

In [734]:
# extract the answer terms from gpt_answer_text
chatgpt_answer_terms = []
for idx, row in test.iterrows():
    answers_text = row['chatgpt_answers_text_DBpedia_2023_03']
    terms = answers_text.replace('https://dbpedia.org/resource/', '').\
    replace('http://dbpedia.org/resource/', '').\
    replace('dbo:', '').strip().lower().split('\n')
    #terms = answers_text.strip().lower().split('\n')
    chatgpt_answer_terms.append([t.strip() for t in terms])

In [184]:
chatgpt_answers

['http://dbpedia.org/resource/Mountain_Time_Zone',
 'http://dbpedia.org/resource/Brutus_the_Younger  \nhttp://dbpedia.org/resource/Gaius_Cassius_Longinus  \nhttp://dbpedia.org/resource/Decimus_Junius_Brutus_Albinus  \nhttp://dbpedia.org/resource/Lucius_Tillius_Cimber',
 'http://dbpedia.org/resource/Zugspitze',
 'http://dbpedia.org/resource/John_F._Kennedy\n    http://dbpedia.org/resource/Lyndon_B._Johnson\n    http://dbpedia.org/resource/Richard_Nixon',
 'http://dbpedia.org/resource/Idaho',
 '8',
 'http://dbpedia.org/resource/Edward_Bawden\nhttp://dbpedia.org/resource/Bruce_Weber\nhttp://dbpedia.org/resource/Amanda_Bynes\nhttp://dbpedia.org/resource/Tamsin_Greig\nhttp://dbpedia.org/resource/Michael_Jeter\nhttp://dbpedia.org/resource/Harold_Pinter\nhttp://dbpedia.org/resource/Kiefer_Sutherland\nhttp://dbpedia.org/resource/Martin_Heidegger\nhttp://dbpedia.org/resource/Antti_Niemi_(footballer,_born_1972)\nhttp://dbpedia.org/resource/Jesus_Castro_(actor,_born_1986)',
 'Science fiction writ

In [735]:
len(chatgpt_answer_terms)

150

In [736]:
import urllib.parse

chatgpt_answer_terms_parsed = []
for terms in chatgpt_answer_terms:
    terms_parsed = []
    for term in terms:
        parsed_string = urllib.parse.unquote(term)
        terms_parsed.append(parsed_string)
    chatgpt_answer_terms_parsed.append(terms_parsed)

In [183]:
chatgpt_answer_terms_parsed

[['mountain_time_zone'],
 ['brutus_the_younger',
  'gaius_cassius_longinus',
  'decimus_junius_brutus_albinus',
  'lucius_tillius_cimber'],
 ['zugspitze'],
 ['john_f._kennedy', 'lyndon_b._johnson', 'richard_nixon'],
 ['idaho'],
 ['8'],
 ['edward_bawden',
  'bruce_weber',
  'amanda_bynes',
  'tamsin_greig',
  'michael_jeter',
  'harold_pinter',
  'kiefer_sutherland',
  'martin_heidegger',
  'antti_niemi_(footballer,_born_1972)',
  'jesus_castro_(actor,_born_1986)'],
 ['science fiction writer.'],
 ['true'],
 ['50,033'],
 ['star_alliance', 'oneworld', 'skyteam'],
 ['andorra',
  '',
  'belgium',
  '',
  'denmark',
  '',
  'liechtenstein',
  '',
  'luxembourg',
  '',
  'monaco',
  '',
  'netherlands',
  '',
  'norway',
  '',
  'sweden',
  '',
  'united_kingdom'],
 ['united_states',
  'spain',
  'china',
  'italy',
  'mexico',
  'france',
  'indonesia',
  'and more...'],
 ['los_angeles_international_airport',
  'san_francisco_international_airport',
  'san_diego_international_airport',
  'oa

In [185]:
test.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers', 'gpt_answers_text',
       'chatgpt_answers_text', 'gold_query_results_03132023',
       'chatgpt_answers_text_DBpedia_03132023'],
      dtype='object')

In [730]:
answer_terms = []
count = 0
for idx, row in test_json_df.iterrows():
    try:
        bindings = row['answers'][0]['results']['bindings']

        answer_list = []
        for item in bindings:
            for k in item:
                answer_list.append(item[k]['value'])

        terms = []
        for ans in answer_list:
            terms.append(ans.replace('http://dbpedia.org/resource/', '').replace('dbo:', '').strip().lower())
        #if terms not in answer_terms:
        answer_terms.append(terms)
              
    except:
        answer_terms.append([str(row['answers'][0]['boolean']).lower()])
        count += 1
        #print(row['answers'])

In [187]:
len(answer_terms)

150

In [113]:
answer_terms

[['mountain_time_zone'],
 ['marcus_junius_brutus_the_younger',
  'gaius_cassius_parmensis',
  'lucius_minucius_basilus',
  'gaius_cassius_longinus',
  'decimus_junius_brutus_albinus',
  'servilius_casca',
  'tillius_cimber',
  'lucius_cornelius_cinna_(suffect_consul)',
  'quintus_antistius_labeo'],
 ['zugspitze'],
 ['john_f._kennedy', 'lyndon_b._johnson', 'richard_nixon'],
 ['idaho'],
 ['8'],
 ['gizem_saka', 'cameron_cartio', 'vesna_pisarović'],
 ['novelist'],
 ['true'],
 ['52000'],
 ['miles_&_more'],
 ['sweden', 'liechtenstein', 'belgium', 'denmark'],
 ['india',
  'philippines',
  'japan',
  'greece',
  'spain',
  'turkey',
  'vietnam',
  'united_states',
  'australia',
  'italy',
  'republic_of_ireland',
  'venezuela',
  'mexico',
  'georgia_(country)',
  'austria',
  'germany',
  'slovenia',
  'gibraltar',
  'serbia',
  'france',
  'abkhazia',
  'jamaica',
  'china',
  'south_africa',
  'brazil',
  'azerbaijan'],
 ['alpine_county_airport',
  'buchanan_field_airport',
  'flabob_airpo

In [741]:
# Evaluate the precision and recall based on the total numbers of 
# gold answers and predicted answers
predicted = 0
gold = 0
predicted_correct = 0
some_matched = {}
pre_gold_lengths = []
for idx, pred_terms in enumerate(chatgpt_answer_terms_parsed):
    gold_terms = answer_terms[idx]
    
    predicted +=  len(pred_terms)
    gold += len(gold_terms)
    
    pre_gold_lengths.append((idx, len(pred_terms), len(gold_terms)))
    
    predicted_correct_idx = False
    for pterm in pred_terms:
        if len(pterm) > 0: # skip an empty string
            for gterm in gold_terms:
                #if pterm ==  gterm:
                pterm = pterm.replace("_", " ")
                gterm = gterm.replace("_", " ")
                if (pterm in gterm) or (gterm in pterm):
                    predicted_correct_idx = True
                    predicted_correct += 1
                    break # this pterm is a correct prediction, skip to next pterm
                          # don't double count this pterm anymore
                
    some_matched[idx] = predicted_correct_idx

In [742]:
pre_gold_lengths

[(0, 1, 1),
 (1, 4, 9),
 (2, 1, 1),
 (3, 3, 3),
 (4, 1, 1),
 (5, 1, 1),
 (6, 10, 3),
 (7, 1, 1),
 (8, 1, 1),
 (9, 1, 1),
 (10, 3, 1),
 (11, 19, 4),
 (12, 8, 26),
 (13, 49, 208),
 (14, 8, 1),
 (15, 1, 1),
 (16, 1, 1),
 (17, 19, 45),
 (18, 1, 2),
 (19, 3, 1),
 (20, 1, 1),
 (21, 5, 2),
 (22, 4, 1),
 (23, 1, 243),
 (24, 1, 1),
 (25, 1, 1),
 (26, 4, 1),
 (27, 1, 1),
 (28, 1, 1),
 (29, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (32, 7, 10),
 (33, 1, 1),
 (34, 1, 1),
 (35, 50, 21),
 (36, 6, 2),
 (37, 1, 1),
 (38, 2, 1),
 (39, 1, 1),
 (40, 1, 1),
 (41, 1, 1),
 (42, 1, 1),
 (43, 55, 2),
 (44, 9, 11),
 (45, 1, 1),
 (46, 1, 1),
 (47, 1, 6),
 (48, 1, 1),
 (49, 1, 1),
 (50, 1, 1),
 (51, 1, 1),
 (52, 20, 915),
 (53, 1, 1),
 (54, 1, 1),
 (55, 1, 1),
 (56, 10, 19),
 (57, 20, 26),
 (58, 2, 1),
 (59, 1, 1),
 (60, 5, 1),
 (61, 1, 1),
 (62, 1, 1),
 (63, 1, 1),
 (64, 1, 2),
 (65, 19, 482),
 (66, 1, 1),
 (67, 1, 1),
 (68, 15, 1),
 (69, 10, 22),
 (70, 11, 15),
 (71, 1, 49),
 (72, 32, 7),
 (73, 9, 13),
 (74, 9, 26),
 

In [738]:
precision = predicted_correct / predicted
precision

0.4415862808145766

In [739]:
recall = predicted_correct/gold
recall

0.08968219416630388

In [740]:
f1 = 2 / (1/precision + 1/recall)
f1

0.14908630360050662

In [ ]:
adj_precision = (predicted_correct-240) / (predicted-240)
adj_recall = (predicted_correct-240)/ (gold - 1714)
adj_f1 = 2 / (1/adj_precision +  1/adj_recall)
print('adj_precision:{},\nadj_recall:{},\nadj_f1:{}'.format(adj_precision, adj_recall, adj_f1))
predicted_correct, predicted, gold

### Use ChatGPT to translate user questions to queries

In [159]:
# For each the test SPARQL query, query the DBpedia endpoint in
# March, 2023

from SPARQLWrapper import SPARQLWrapper, JSON

# set up the SPARQL endpoint URL
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

sparql.setReturnFormat(JSON)

gold_query_results = []
count = 0
for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    #if count > 3:
    #    break
     
    count += 1
    gold_query = row['sparql_query']
    
    sparql.setQuery(gold_query)
    
    try:
        ret = sparql.queryAndConvert()

        gold_query_results.append(ret)
    except Exception as e:
        gold_query_results.append('ERROR')
        print(e)

100%|██████████| 150/150 [02:39<00:00,  1.06s/it]


In [160]:
gold_query_results

[{'head': {'link': [], 'vars': ['uri']},
  'results': {'distinct': False,
   'ordered': True,
   'bindings': [{'uri': {'type': 'uri',
      'value': 'http://dbpedia.org/resource/Mountain_Time_Zone'}}]}},
 {'head': {'link': [], 'vars': ['uri']},
  'results': {'distinct': False,
   'ordered': True,
   'bindings': [{'uri': {'type': 'uri',
      'value': 'http://dbpedia.org/resource/Decimus_Junius_Brutus_Albinus'}},
    {'uri': {'type': 'uri',
      'value': 'http://dbpedia.org/resource/Gaius_Cassius_Longinus'}},
    {'uri': {'type': 'uri',
      'value': 'http://dbpedia.org/resource/Marcus_Junius_Brutus'}},
    {'uri': {'type': 'uri',
      'value': 'http://dbpedia.org/resource/Pontius_Aquila'}},
    {'uri': {'type': 'uri',
      'value': 'http://dbpedia.org/resource/Pacuvius_Labeo'}},
    {'uri': {'type': 'uri',
      'value': 'http://dbpedia.org/resource/Quintus_Ligarius'}},
    {'uri': {'type': 'uri',
      'value': 'http://dbpedia.org/resource/Publius_Servilius_Casca'}},
    {'uri': {

In [161]:
test['gold_query_results_03132023'] = gold_query_results

In [162]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

In [85]:
# ChatGPT translate user questions to SPARQL Queries directly

chatgpt_queries = []
count = 0
for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    #if count > 3:
    #    break
     
    count += 1
    question_text = row['question_text']
    keywords = row['question_keywords']
    
    answertype = row['answertype']
    answertype_text = ""
    if answertype == 'resource':
        answertype_text = 'DBpedia Resource URI(s)'
    else:
        answertype_text = answertype
    
    answeragg = row['aggregation']
    answeragg_text = "The answers do not need aggregation"
    if answeragg:
        answeragg_text = "The answers need aggregration"
        
    msg = """
        Translate the following question to SPARQL query on the 
        DBpedia knowledge base. The output query should include 
        all necessary prefixes for querying the current DBpedia endpoint. 
        No comments. Output SPARQL query only.
        The query should return answers as {}.
        
        QUESTION: {} 
        
        QUERY: 
    """
    
    msg = msg.format(answertype_text, question_text)
    
    #print(msg)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant focusing on \
             DBpedia. You will translate user questions to SPARQL queries on \
             the current DBpedia knowledge base."},
            {"role": "user", "content": msg}
        ]
    )

    chatgpt_queries.append(response['choices'][0]['message']['content'])

100%|██████████| 150/150 [11:32<00:00,  4.62s/it]


In [86]:
chatgpt_queries

['PREFIX dbo: <http://dbpedia.org/ontology/>\nPREFIX dbp: <http://dbpedia.org/property/>\n\nSELECT ?timezone WHERE {\n  ?city dbo:country <http://dbpedia.org/resource/United_States> .\n  ?city dbp:name "Salt Lake City"@en .\n  ?city dbp:timezone ?timezone .\n}',
 'PREFIX dbpedia: <http://dbpedia.org/resource/>\n        PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>\n        \n        SELECT ?killer WHERE {\n          dbpedia:Julius_Caesar dbpedia-owl:deathCause ?cause .\n          ?event dbpedia-owl:death ?cause .\n          ?event dbpedia-owl:responsibleFor ?killer .\n        }',
 '\n\n        PREFIX dbo: <http://dbpedia.org/ontology/>\n        PREFIX dbr: <http://dbpedia.org/resource/>\n        PREFIX dbp: <http://dbpedia.org/property/>\n        \n        SELECT DISTINCT ?mountain\n        WHERE {\n            ?mountain rdf:type dbo:Mountain .\n            ?mountain dbo:country dbr:Germany .\n            ?mountain dbo:elevation ?elevation .\n        }\n        ORDER BY DESC(?elev

In [87]:
chatgpt_queries_text = [" ".join(item.split('\n')).strip() for item in chatgpt_queries]

In [88]:
test['chatgpt_query_DBpedia_2023_03'] = chatgpt_queries_text

In [89]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

In [90]:
test.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers', 'gpt_answers_text_DBpedia-2016-04',
       'chatgpt_answers_text_DBpedia_2016_04',
       'gold_query_results_DBpedia_2023_03',
       'chatgpt_answers_text_DBpedia_2023_03', 'gpt_query_DBpedia_2023_03',
       'gpt_query_results_DBpedia_2023_03', 'chatgpt_query_DBpedia_2023_03'],
      dtype='object')

#### Retrieve chatgpt_query_results_DBpedia_2023_03

In [95]:
test = pd.read_csv('../data/QALD/9/data/qald-9-test.csv')
test.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers', 'gpt_answers_text_DBpedia-2016-04',
       'chatgpt_answers_text_DBpedia_2016_04',
       'gold_query_results_DBpedia_2023_03',
       'chatgpt_answers_text_DBpedia_2023_03', 'gpt_query_DBpedia_2023_03',
       'gpt_query_results_DBpedia_2023_03', 'chatgpt_query_DBpedia_2023_03'],
      dtype='object')

In [96]:
# Retrieve the chatgpt_query_results
# For each the test GPT query, query the DBpedia endpoint in
# March, 2023

from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import ast

# set up the SPARQL endpoint URL
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

sparql.setReturnFormat(JSON)

chatgpt_query_results = []
count = 0
for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    #if count > 3:
    #   break
     
    count += 1
    chatgpt_query = row['chatgpt_query_DBpedia_2023_03']
    
    sparql.setQuery(chatgpt_query)
    
    try:
        ret = sparql.queryAndConvert()

        chatgpt_query_results.append(ret)
    except Exception as e:
        chatgpt_query_results.append(e)
        print(e)

 27%|██▋       | 40/150 [01:50<06:42,  3.66s/it]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at \';\' before \'}\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/>     PREFIX dbp: <http://dbpedia.org/property/>     PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>     SELECT ?state WHERE {         ?state rdf:type dbo:AdministrativeRegion;             dbo:populationDensity ?density.         FILTER regex(str(?state), "http://dbpedia.org/resource/United_States_");     } ORDER BY DESC(?density)     LIMIT 1.\n'


 59%|█████▉    | 89/150 [04:47<04:00,  3.94s/it]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 2: Undefined namespace prefix in prefix:localpart notation at \'geof:distance\' before \'(\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> SELECT DISTINCT ?country  WHERE {   ?river dbo:riverName "Rhine"@en .   ?river dbo:mouth ?mouth .   ?mouth geo:lat ?lat ;          geo:long ?long .   ?border dbo:point ?point .   ?border dbo:country ?country .   FILTER(geof:distance(?point, "POINT(?long ?lat)") < 50000) }\n'


 73%|███████▎  | 109/150 [05:38<01:15,  1.83s/it]

...

 91%|█████████ | 136/150 [07:04<00:50,  3.59s/it]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at '}' before 'ORDER'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/>  SELECT DISTINCT ?company WHERE {   ?company rdf:type dbo:Company ;            dbo:location dbr:North_Rhine-Westphalia ;            dbo:product dbo:Beer ; } ORDER BY ?company\n"


 91%|█████████▏| 137/150 [07:11<01:00,  4.63s/it]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at ',' before '_Princess_of_Wales'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbpedia: <http://dbpedia.org/resource/> PREFIX dbp: <http://dbpedia.org/ontology/>  SELECT ?date WHERE {   dbpedia:Diana,_Princess_of_Wales dbp:dateOfDeath ?date . }\n"


100%|██████████| 150/150 [08:21<00:00,  3.34s/it]


In [97]:
len(chatgpt_query_results)

150

In [98]:
test['chatgpt_query_results_DBpedia_2023_03'] = chatgpt_query_results

In [99]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

In [100]:
test.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers', 'gpt_answers_text_DBpedia-2016-04',
       'chatgpt_answers_text_DBpedia_2016_04',
       'gold_query_results_DBpedia_2023_03',
       'chatgpt_answers_text_DBpedia_2023_03', 'gpt_query_DBpedia_2023_03',
       'gpt_query_results_DBpedia_2023_03', 'chatgpt_query_DBpedia_2023_03',
       'chatgpt_query_results_DBpedia_2023_03'],
      dtype='object')

### Use ChatGPT to translate user questions to queries by few-shot learning

In [156]:
train = pd.read_csv('../data/QALD/9/data/qald-9-train-with-embeddings-cot.csv')
train.head()

,id,answertype,aggregation,onlydbo,hybrid,question_text,question_keywords,sparql_query,answer_head,answer_results,question,query,answers
0,1,resource,False,True,False,List all boardgames by GMT.,"boardgame, GMT",PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Liste alle Bret...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
1,2,resource,False,True,False,Who developed Skype?,"develop, Skype",PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Wer entwickelt ...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
2,3,resource,False,False,False,Which people were born in Heraklion?,"people, born, heraklion",PREFIX yago: <http://dbpedia.org/class/yago/> ...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Welche Menschen...",{'sparql': 'PREFIX yago: <http://dbpedia.org/c...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
3,4,resource,False,True,False,In which U.S. state is Area 51 located?,"Area 51, located, U.S. state",PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Im welche US Zu...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."
4,5,resource,False,True,False,Who is the mayor of New York City?,"New York City, mayor",PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,{'vars': ['uri']},"{'bindings': [{'uri': {'type': 'uri', 'value':...","[{'language': 'de', 'string': 'Wer ist der Bür...",{'sparql': 'PREFIX dbo: <http://dbpedia.org/on...,"[{'head': {'vars': ['uri']}, 'results': {'bind..."


In [158]:
train.shape

(408, 13)

In [157]:
test = pd.read_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv')
test.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers', 'gpt_answers_text_DBpedia-2016-04',
       'chatgpt_answers_text_DBpedia_2016_04',
       'gold_query_results_DBpedia_2023_03',
       'chatgpt_answers_text_DBpedia_2023_03', 'gpt_query_DBpedia_2023_03',
       'gpt_query_results_DBpedia_2023_03', 'chatgpt_query_DBpedia_2023_03',
       'chatgpt_query_results_DBpedia_2023_03'],
      dtype='object')

#### Embed Train Questions

In [159]:
# imports

import tiktoken

from openai.embeddings_utils import get_embedding

In [160]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [211]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
train_embeddings = []
for idx, row in tqdm(train.iterrows(), total=train.shape[0]):
    question_text = row['question_text']
    
    train_embeddings.append(get_embedding(question_text, engine=embedding_model))

100%|██████████| 408/408 [01:37<00:00,  4.17it/s]


In [212]:
train['train_question_embedding'] = train_embeddings

In [213]:
train.iloc[9].train_question_embedding

[-0.003097536275163293,
 0.003752992721274495,
 -0.005736407823860645,
 -0.02853638119995594,
 -0.0044688815250992775,
 -0.0052312565967440605,
 -0.00836753100156784,
 -0.0028310148045420647,
 -0.024309229105710983,
 -0.025214161723852158,
 0.0013860660837963223,
 0.02454475872218609,
 -0.0021895039826631546,
 0.006464692763984203,
 -0.021606825292110443,
 -0.0035081650130450726,
 0.005513273645192385,
 -0.02522655762732029,
 0.012303370051085949,
 -0.010636062361299992,
 -0.011057537980377674,
 0.028561173006892204,
 0.01977216638624668,
 -0.006725016050040722,
 -0.017578013241291046,
 0.024730704724788666,
 0.009861290454864502,
 -0.022586137056350708,
 0.0023367106914520264,
 -0.02672651596367359,
 0.022548947483301163,
 0.004682718310505152,
 0.007258058525621891,
 -0.007096906192600727,
 -0.018507739529013634,
 -0.02130931243300438,
 0.008119604550302029,
 -0.002342908876016736,
 -0.01521031279116869,
 8.407480891037267e-06,
 0.03557750582695007,
 0.00733863515779376,
 0.020540740

In [214]:
#train.to_csv('../data/QALD/9/data/qald-9-train-with-embeddings-cot.csv', index=None)

In [217]:
# Load the question embeddings
import pandas as pd
import numpy as np

datafile_path = "../data/QALD/9/data/qald-9-train.csv"

train = pd.read_csv(datafile_path)
train["train_question_embedding"] = train.train_question_embedding.apply(eval).apply(np.array)

#### Embed test question and search similar train questions

In [24]:
from openai.embeddings_utils import get_embedding, cosine_similarity

In [ ]:
def search_question_query(df, input_question, n=3):
    input_question_embedding = get_embedding(
        input_question,
        engine="text-embedding-ada-002"
    )
    df["similarity"] = df.train_question_embedding.apply(lambda x: cosine_similarity(x, input_question_embedding))

    results = \
        df.sort_values("similarity", ascending=False) \
        .head(n)[['question_text', 'sparql_query']]
    
    return input_question_embedding, list(results['question_text'].values), list(results['sparql_query'].values)

In [310]:
test_question_embeddings = []
for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    test_question = row['question_text']
    question_embedding = get_embedding(
        test_question,
        engine="text-embedding-ada-002"
    )
    test_question_embeddings.append(question_embedding)

100%|██████████| 150/150 [00:22<00:00,  6.76it/s]


In [313]:
test['test_question_embedding'] = test_question_embeddings

In [314]:
#test = pd.read_csv('../data/QALD/9/data/qald-9-test.csv')
#test["test_question_embedding"] = test.test_question_embedding.apply(eval).apply(np.array)

In [198]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

In [219]:
train = pd.read_csv('../data/QALD/9/data/qald-9-train-with-embeddings-cot.csv')

In [220]:
train["train_question_embedding"] = train.train_question_embedding.apply(eval).apply(np.array)

In [199]:
#train.to_csv('../data/QALD/9/data/qald-9-train-with-embeddings-cot.csv', index=None)

In [313]:
from openai.embeddings_utils import get_embedding, cosine_similarity

train_matched_questions = []
train_matched_queries = []
for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    test_question_embedding = row['test_question_embedding']
    
    train_embeddings_similarities = train_embeddings.question_embedding.apply(lambda x: cosine_similarity(x,test_question_embedding))
    
    max_idx = train_embeddings_similarities.idxmax()
    
    results = \
        train_embeddings.iloc[max_idx][['question', 'query']]
    
    matched_question = results['question']
    matched_query = results['query']

    train_matched_questions.append(matched_question)
    train_matched_queries.append(matched_query)

100%|██████████| 129/129 [00:24<00:00,  5.32it/s]


#### Few-shot learning on matched train question only

In [560]:
from tqdm import tqdm

chatgpt_train_fewshot_query = []

count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    
    if count > 4:
        break
    count += 1
    
    test_question = row['question_text']

    test_question_embedding = row['test_question_embedding']
    
    train_question_idx = train.index[train['question_text'] == test_question].tolist()
    
    if len(train_question_idx) > 0: # find a matched train question, skip to generate new query
        chatgpt_train_fewshot_query.append(train.iloc[train_question_idx[0]]['sparql_query'])
    
    else:

        train_embeddings_similarities = train.train_question_embedding.\
        apply(lambda x: cosine_similarity(x,test_question_embedding))

        top5_idx = train_embeddings_similarities.sort_values(ascending=False)[:5].index

        top5_train_questions = list(train.iloc[top5_idx].question_text.values)
        top5_train_queries = list(train.iloc[top5_idx].sparql_query.values)

        sys_question_query = """
            Learn the following example question and corresponding query.
            
            Question: {} 
            Query: {}
        """
        sys_question_query = sys_question_query.format(top5_train_questions[0], 
                                                  top5_train_queries[0])
    
        msg = """
           Only use the terms defined in the DBpedia ontology. 
           Translate the following question to SPARQL query. 
           Output query only. No comments. 
           Output syntactically correct query only. 
           Some common prefixes can be used in the query:
            PREFIX dbo: <http://dbpedia.org/ontology/>
            PREFIX dbr: <http://dbpedia.org/resource/>
            PREFIX dbc: <http://dbpedia.org/resource/Category:>
            PREFIX foaf: <http://xmlns.com/foaf/0.1/>
            PREFIX dc: <http://purl.org/dc/elements/1.1/>
            PREFIX dct: <http://purl.org/dc/terms/>
        
           Question:{}
           Query:
        """
        msg = msg.format(test_question)

        response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "you are a helpful assistant focusing on DBpedia. \
            You will translate a user question to a SPARQL on the DBpedia knowledge base."},
            {"role": "system", "content": sys_question_query},
            {"role": "user", "content": msg}
            ]
        )


        chatgpt_train_fewshot_query.append(response['choices'][0]['message']['content'].\
                                   strip().replace('\n', ' ' ))

  3%|▎         | 5/150 [00:22<10:55,  4.52s/it]


In [561]:
chatgpt_train_fewshot_query

['SELECT ?timezone WHERE {     dbr:Salt_Lake_City dbo:timeZone ?timezone . }',
 'SELECT ?assassin WHERE {   dbr:Caesar dbo:deathCause ?cause ;             dbo:deathPlace ?place ;             dbo:deathYear ?year ;             dbo:deathDate ?date    FILTER ((regex(?cause, "assassinat", "i")) || (regex(?place, "theatre of Pompey", "i")))   ?event dbo:agent ?assassin .   ?assassin a dbo:Person .   FILTER NOT EXISTS {?assassin dbo:title ?title .} } LIMIT 10',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> SELECT DISTINCT ?m WHERE {     ?m rdf:type dbo:Mountain ;        dct:subject dbc:Mountains_of_Germany ;        dbo:elevation ?h .  } ORDER BY DESC(?h) LIMIT 1.',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> PREFIX dbc: <http://dbpedia.org/resource/Category:> PREFIX foaf: <http://xmlns.com/foaf/0.1/> PREFIX dc: <http://purl.org/dc/elements/1.1/> PREFIX dct: <http://purl.org/dc/terms/>  SELECT DISTINCT ?presi

In [424]:
test.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers', 'gpt_answers_text_DBpedia-2016-04',
       'chatgpt_answers_text_DBpedia_2016_04',
       'gold_query_results_DBpedia_2023_03',
       'chatgpt_answers_text_DBpedia_2023_03', 'gpt_query_DBpedia_2023_03',
       'gpt_query_results_DBpedia_2023_03', 'chatgpt_query_DBpedia_2023_03',
       'chatgpt_query_results_DBpedia_2023_03', 'test_question_embedding',
       'chatgpt_train_3fewshot_query', 'chatgpt_train_3fewshot_query_results',
       'chatgpt_train_1fewshot_query', 'chatgpt_train_1fewshot_query_results'],
      dtype='object')

In [478]:
len(chatgpt_train_fewshot_query)

150

In [479]:
chatgpt_train_fewshot_query

['SELECT ?timezone WHERE {   <http://dbpedia.org/resource/Salt_Lake_City> <http://dbpedia.org/ontology/timeZone> ?timezone . }',
 'SELECT DISTINCT ?killer WHERE {     ?assassination <http://dbpedia.org/ontology/target> <http://dbpedia.org/resource/Julius_Caesar> .     ?assassination <http://dbpedia.org/ontology/motive> <http://dbpedia.org/resource/Assassination_of_Julius_Caesar> .     ?assassination <http://dbpedia.org/ontology/perpetrator> ?killer .     ?killer a <http://dbpedia.org/ontology/Person> . }',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> SELECT ?m WHERE {    ?m a dbo:Mountain ;       dbo:locatedInArea dbr:Germany ;       dbo:elevation ?height . } ORDER BY DESC(?height) LIMIT 1',
 'SELECT DISTINCT ?president WHERE {     ?president a <http://dbpedia.org/ontology/President> ;                <http://dbpedia.org/ontology/office> ?office ;                dct:subject <http://dbpedia.org/resource/Category:Presidents_of_the_United_States> 

In [480]:
test['chatgpt_train_1fewshot_query'] = chatgpt_train_fewshot_query

#### Save train and test embeddings narry in Disk

In [381]:
#train[['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
#       'question_keywords', 'sparql_query', 'train_question_embedding']].\
#to_csv('../data/QALD/9/data/qald-9-train-with-embeddings-cot.csv', index=None)

In [384]:
#test[['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
#       'question_keywords', 'sparql_query', 'test_question_embedding']].\
#       to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

#### Retrieve chatgpt_train_fewshot_query results

In [646]:
# Retrieve the chatgpt_train_3fewshot_query_results
# query the DBpedia endpoint in
# March, 2023

from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import ast

# set up the SPARQL endpoint URL
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

sparql.setReturnFormat(JSON)

chatgpt_train_fewshot_query_results = []
count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    #if count > 3:
    #    break
     
    count += 1
    chatgpt_train_fewshot_query = row['chatgpt_train_1fewshot_query']
    
    sparql.setQuery(chatgpt_train_fewshot_query)
    
    try:
        ret = sparql.queryAndConvert()

        chatgpt_train_fewshot_query_results.append(ret)
    except Exception as e:
        chatgpt_train_fewshot_query_results.append(e)
        print(e)

  3%|▎         | 4/150 [00:03<02:04,  1.17it/s]

EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Virtuoso S0022 Error SQ200: No column t18.S.\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT DISTINCT ?president WHERE {     ?president a <http://dbpedia.org/ontology/President> ;                <http://dbpedia.org/ontology/office> ?office ;                dct:subject <http://dbpedia.org/resource/Category:Presidents_of_the_United_States> .     ?office <http://dbpedia.org/ontology/startDate> ?start_date .     FILTER (?start_date <= "1975-04-30"^^xsd:date )     FILTER NOT EXISTS {         ?president <http://dbpedia.org/ontology/predecessor> ?predecessor .         ?predecessor <http://dbpedia.org/ontology/office> ?predecessor_office .         ?predecessor_office <http://dbpedia.org/ontology/startDate> ?predecessor_start_date .         FILTER (?predecessor_start_date <= "1975-04-30"^^xsd:date )     }     FILTER NOT EXISTS {         ?successor <http://dbpedia.org/ontology/predecesso

  9%|▉         | 14/150 [00:14<01:51,  1.22it/s]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at 'California' before '.'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbp: <http://dbpedia.org/property/> PREFIX res: <http://dbpedia.org/resource/>  SELECT DISTINCT ?airport WHERE {     ?airport rdf:type dbo:Airport .     ?airport dbo:location ?state .     ?state dbp:state California .     ?state dbo:country res:United_States .  }\n"


 16%|█▌        | 24/150 [00:21<01:38,  1.28it/s]

...

 91%|█████████▏| 137/150 [01:46<00:08,  1.56it/s]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at ',' before '_Princess_of_Wales'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> SELECT ?x WHERE {   dbr:Diana,_Princess_of_Wales dbo:dateOfDeath ?x } LIMIT 1\n"


 92%|█████████▏| 138/150 [01:47<00:07,  1.58it/s]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 3: syntax error at '}'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/>     PREFIX res: <http://dbpedia.org/resource/>     SELECT DISTINCT ?founder WHERE {         res:Intel dbo:fo\n"


100%|██████████| 150/150 [01:55<00:00,  1.30it/s]


In [647]:
test.iloc[125].sparql_query

'PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX foaf: <http://xmlns.com/foaf/0.1/> PREFIX onto: <http://dbpedia.org/ontology/> SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Charmed> onto:starring ?actors . ?actors foaf:homepage ?uri }'

In [484]:
chatgpt_train_fewshot_query_results

[{'head': {'link': [], 'vars': ['timezone']},
  'results': {'distinct': False,
   'ordered': True,
   'bindings': [{'timezone': {'type': 'uri',
      'value': 'http://dbpedia.org/resource/Mountain_Time_Zone'}}]}},
 {'head': {'link': [], 'vars': ['killer']},
  'results': {'distinct': False, 'ordered': True, 'bindings': []}},
 {'head': {'link': [], 'vars': ['m']},
  'results': {'distinct': False,
   'ordered': True,
   'bindings': [{'m': {'type': 'uri',
      'value': 'http://dbpedia.org/resource/Zugspitze'}}]}},
 SPARQLWrapper.SPARQLExceptions.EndPointInternalError('EndPointInternalError: The endpoint returned the HTTP status code 500. \n\nResponse:\nb\'Virtuoso S0022 Error SQ200: No column t18.S.\\n\\nSPARQL query:\\n#output-format:application/sparql-results+json\\nSELECT DISTINCT ?president WHERE {     ?president a <http://dbpedia.org/ontology/President> ;                <http://dbpedia.org/ontology/office> ?office ;                dct:subject <http://dbpedia.org/resource/Category:Pre

In [648]:
test['chatgpt_train_1fewshot_query_results'] = chatgpt_train_fewshot_query_results

In [649]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

In [390]:
test.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers', 'gpt_answers_text_DBpedia-2016-04',
       'chatgpt_answers_text_DBpedia_2016_04',
       'gold_query_results_DBpedia_2023_03',
       'chatgpt_answers_text_DBpedia_2023_03', 'gpt_query_DBpedia_2023_03',
       'gpt_query_results_DBpedia_2023_03', 'chatgpt_query_DBpedia_2023_03',
       'chatgpt_query_results_DBpedia_2023_03', 'test_question_embedding',
       'chatgpt_train_3fewshot_query', 'chatgpt_train_3fewshot_query_results',
       'chatgpt_train_1fewshot_query', 'chatgpt_train_1fewshot_query_results'],
      dtype='object')

### Use ChatGPT to translate user questions to queries by few-shot learning WITH MASKED ENTITIEs

#### First, mask entities in train and test questions

In [954]:
from tqdm import tqdm

masked_results = []

count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    
    #if count > 20:
    #    break
    #count += 1
    
    if (idx > -1) and (idx < 200):
    #if idx < 3:
        t_question = row['question_text']
        t_query = row['sparql_query']

        question_keywords = row['question_keywords']

        msg = """
            Identify any named entities in the question and replace 
            the discovered named entities with [MASK1], [MASK2], etc.
            One mask for each named entity only.
            Merge consecutive masks into one mask.
            No comments. Output masked question only.
           
            Question:{}
            Masked Question:
        """
        msg = msg.format(t_question)

        response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "you are a helpful assistant focusing on\
                identifying named entities and replacing them with masks."},
                {"role": "user", "content": msg}
                ]
        )


        masked_results.append(response['choices'][0]['message']['content'])
                                   #strip().replace('\n', ' ' ))

100%|██████████| 150/150 [07:08<00:00,  2.85s/it]


In [955]:
masked_results

['What is the time zone of [MASK1]?',
 'Who killed [MASK1]?',
 'What is the highest mountain in [MASK1]?',
 'Which [MASK1] were in office during the Vietnam War?',
 '[MASK1] is the governor of which U.S. state?',
 'How many gold medals did [MASK1] win at the [MASK2] [MASK3]?',
 'Which artists were born on the same date as [MASK1]?',
 'What is the profession of [MASK1]?',
 'Are [MASK1] some kind of Japanese musical instrument?',
 'How many seats does the home stadium of [MASK1] [MASK2] have?',
 'Which frequent flyer program has the most airlines?',
 'Which [MASK1] countries have a constitutional monarchy?',
 'Which countries have places with more than two [MASK1]?',
 'Which airports are located in [MASK1], [MASK2]?',
 'What are the nicknames of [MASK1]?',
 'What is [MASK1] birth name? \n\nMasked Named Entities: \n[MASK1] - Angela Merkel',
 'Who is the mayor of [MASK1]?',
 'Which countries in the [MASK1] [MASK2] adopted the Euro?',
 'Which software has been published by [MASK1]?',
 'Whic

In [956]:
#with open('../data/QALD/9/data/test_masked_questions.txt', 'w') as file:
#    for item in masked_results:
#        file.write(item + '\n')

In [957]:
masked_cleaned = []
with open('../data/QALD/9/data/test_masked_questions.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        masked_cleaned.append(line.strip())
masked_cleaned

['What is the time zone of [MASK1]?',
 'Who killed [MASK1]?',
 'What is the highest mountain in [MASK1]?',
 'Which [MASK1] were in office during the Vietnam War?',
 '[MASK1] is the governor of which U.S. state?',
 'How many gold medals did [MASK1] win at the [MASK2] [MASK3]?',
 'Which artists were born on the same date as [MASK1]?',
 'What is the profession of [MASK1]?',
 'Are [MASK1] some kind of Japanese musical instrument?',
 'How many seats does the home stadium of [MASK1] have?',
 'Which frequent flyer program has the most airlines?',
 'Which [MASK1] countries have a constitutional monarchy?',
 'Which countries have places with more than two [MASK1]?',
 'Which airports are located in [MASK1], [MASK2]?',
 'What are the nicknames of [MASK1]?',
 'What is [MASK1] birth name?',
 'Who is the mayor of [MASK1]?',
 'Which countries in the [MASK1] adopted the Euro?',
 'Which software has been published by [MASK1]?',
 'Which country was [MASK1] born in?',
 'How many grand-children did [MASK1

In [958]:
len(masked_cleaned)

150

In [959]:
test['masked_question'] = masked_cleaned

In [960]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

In [1101]:
from tqdm import tqdm

masked_results = []

count = 0

for idx, row in tqdm(train.iterrows(), total=train.shape[0]):
    
    #if count > 20:
    #    break
    #count += 1
    
    if (idx > 381 ) and (idx < 383):
    #if idx < 3:
        t_question = row['question_text']
        t_query = row['sparql_query']

        question_keywords = row['question_keywords']

        msg = """
            Identify any named entities in the SPARQL query and replace 
            the identified named entities with [MASK1], [MASK2], etc.
            One mask for each named entity only.
            Merge consecutive masks into one mask.
            Output masked query only. No comments. 
           
            Query:{}
            Masked Query:
        """
        msg = msg.format(t_query)

        response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "you are a helpful assistant focusing on\
                identifying named entities and replacing them with masks."},
                {"role": "user", "content": msg}
                ]
        )


        masked_results.append(response['choices'][0]['message']['content'])
                                   #strip().replace('\n', ' ' ))

100%|██████████| 408/408 [00:02<00:00, 194.62it/s]


In [1102]:
masked_results

['PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?d WHERE { res:[MASK1] dbo:activeYearsEndDate ?d }']

In [973]:
len(masked_results)

50

In [974]:
#masked_20 = []
for item in masked_results:
    masked_20.append(item)
masked_20

['PREFIX res: <http://dbpedia.org/resource/> PREFIX dbp: <http://dbpedia.org/property/> SELECT DISTINCT ?uri WHERE { res:[MASK1] <http://dbpedia.org/ontology/timeZone> ?uri }',
 'PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT DISTINCT ?uri WHERE { ?uri dct:subject dbc:[MASK1] }',
 'PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX onto: <http://dbpedia.org/ontology/> SELECT ?uri WHERE { ?uri rdf:type onto:Mountain ; onto:elevation [MASK1] ; onto:locatedInArea [MASK2] } ORDER BY DESC(?elevation) LIMIT 1',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT ?uri WHERE { ?uri dct:subject dbc:[MASK1] . res:[MASK2] dbo:commander ?uri }',
 'SELECT DISTINCT ?uri WHERE { ?uri a [MASK1] ; [MASK2] [MASK3] }',
 'PREFIX dbo: <http://dbpedia.org/on

In [975]:
len(masked_20)

150

In [976]:
#with open('../data/QALD/9/data/test_masked_queries.txt', 'w') as file:
#    for item in masked_20:
#        file.write(item + '\n')

In [1103]:
masked_cleaned = []
with open('../data/QALD/9/data/train_masked_queries.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        if len(line) > 0:
            masked_cleaned.append(line.strip())
masked_cleaned

['PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?uri WHERE { ?uri dbo:publisher [MASK1]}',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT DISTINCT ?uri WHERE { [MASK1] dbo:developer [MASK2]. }',
 'PREFIX yago: <http://dbpedia.org/class/yago/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX onto: <http://dbpedia.org/ontology/> SELECT DISTINCT [MASK1] WHERE { [MASK1] rdf:type onto:Person ; onto:birthPlace [MASK2]. }',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT DISTINCT ?uri WHERE { [MASK1] dbo:location [MASK2] . [MASK2] dbo:country [MASK1]. }',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.or

In [1104]:
len(masked_cleaned)

408

In [1105]:
train['masked_query'] = masked_cleaned

In [1106]:
#train.to_csv('../data/QALD/9/data/qald-9-train-with-embeddings-cot.csv', index=None)

### Get Chain of Thought for train and test masked queries

In [1124]:
from tqdm import tqdm

masked_cot = []

count = 0

for idx, row in tqdm(train.iterrows(), total=train.shape[0]):
    
    #if count > 4:
    #    break
    count += 1
    
    if (idx > 381) and (idx < 383):
    #if idx < 3:
        m_query = row['masked_query']


        msg = """
               Briefly explain the following query in logical steps as a chain of thought. 
               Explain in natural language. 
               Forget what you have about the query before. 
               Assume you are trying to construct the query again.
               Treat the mask variables, [MASK1], [MASK2] as real entities.
               No comments. Output the steps only. 
               Do not include the original query in the explanation. 

               QUERY:{}
               THOUGHT:
        """
        msg = msg.format(m_query)

        response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "you are a helpful assistant focusing on DBpedia. \
                 You will explain a masked SPARQL query in logical steps to help reconstruct the query. "},
                {"role": "user", "content": msg}
                ]
        )


        masked_cot.append(response['choices'][0]['message']['content'].\
                                       strip().replace('\n', ' ' ))

100%|██████████| 408/408 [00:05<00:00, 69.04it/s]


In [1052]:
len(masked_cot)

158

In [1128]:
train['masked_cot']=masked_cot_fix

In [1129]:
#train.to_csv('../data/QALD/9/data/qald-9-train-with-embeddings-cot.csv', index=None)

In [1153]:
from tqdm import tqdm

masked_cot = []

count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    
    #if count > 4:
    #    break
    count += 1
    
    if (idx > 99) and (idx < 200):
    #if idx < 3:
        m_query = row['masked_query']


        msg = """
               Briefly explain the following query in logical steps as a chain of thought. 
               Explain in natural language. 
               Forget what you have about the query before. 
               Assume you are trying to construct the query again.
               Treat the mask variables, [MASK1], [MASK2] as real entities.
               No comments. Output the steps only. 
               Do not include the original query in the explanation. 

               QUERY:{}
               THOUGHT:
        """
        msg = msg.format(m_query)

        response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "you are a helpful assistant focusing on DBpedia. \
                 You will explain a masked SPARQL query in logical steps to help reconstruct the query. "},
                {"role": "user", "content": msg}
                ]
        )


        masked_cot.append(response['choices'][0]['message']['content'].\
                                       strip().replace('\n', ' ' ))

100%|██████████| 150/150 [04:54<00:00,  1.96s/it]


In [1151]:
masked_cot

['1. Set the prefixes for DBpedia ontology, XML Schema Datatypes, and DBpedia resources. 2. Select the variable ?years. 3. Specify the WHERE clause. 4. Find a DBpedia resource. 5. Check if it has a production end year and production start year using the DBpedia ontology dbo:productionEndYear and dbo:productionStartYear predicates. 6. Retrieve the values of ?end and ?start for the resource. 7. Use the year() function with the xsd:date datatype to extract only the year from the date values of ?end and ?start. 8. Subtract the year values of ?start from ?end. 9. Bind the result of the subtraction operation as ?years. 10. Output the values of ?years.',
 '1. We are querying DBpedia. 2. We want to retrieve a specific type of information from the database. 3. We need to specify the prefixes for the entities we will use in the query. 4. We only want to retrieve distinct dates. 5. We are interested in the date of a particular event related to an entity in DBpedia. 6. The entity we are interested

In [1156]:
test['masked_cot']=masked_cot_50

In [1157]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

### Embed Train Masked Questions

In [985]:
# imports

import tiktoken

from openai.embeddings_utils import get_embedding

In [986]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [987]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
train_masked_embeddings = []
for idx, row in tqdm(train.iterrows(), total=train.shape[0]):
    question_text = row['masked_question']
    
    train_masked_embeddings.append(get_embedding(question_text, engine=embedding_model))

100%|██████████| 408/408 [01:23<00:00,  4.86it/s]


In [988]:
train['train_masked_question_embedding'] = train_masked_embeddings

In [990]:
#train.to_csv('../data/QALD/9/data/qald-9-train-with-embeddings-cot.csv', index=None)

### Embed test question and search similar train questions

In [991]:
from openai.embeddings_utils import get_embedding, cosine_similarity

In [1004]:
# search through the train for matched question
def search_question(df, input_question, n=3, pprint=True, masked=False):
    input_question_embedding = get_embedding(
        input_question,
        engine="text-embedding-ada-002"
    )
    if masked:
        df["similarity"] = df.train_masked_question_embedding.apply(lambda x: cosine_similarity(x, input_question_embedding))
    else:
        df["similarity"] = df.train_question_embedding.apply(lambda x: cosine_similarity(x, input_question_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)[['question_text', 'masked_question']]
    )
    
    
    #if pprint:
    #    for r in results:
    #        print(r[:200])
    #        print()
    return results


In [1010]:
def search_question_query(df, input_question, n=3, masked=False):
    input_question_embedding = get_embedding(
        input_question,
        engine="text-embedding-ada-002"
    )
    if masked:
        df["similarity"] = df.train_masked_question_embedding.apply(lambda x: cosine_similarity(x, input_question_embedding))
    else:
        df["similarity"] = df.train_question_embedding.apply(lambda x: cosine_similarity(x, input_question_embedding))

    results = \
        df.sort_values("similarity", ascending=False) \
        .head(n)[['question_text', 'sparql_query', 'masked_question', 'masked_query']]
    
    return input_question_embedding, list(results['question_text'].values), list(results['sparql_query'].values),\
                  list(results['masked_question'].values), list(results['masked_query'].values)

In [310]:
test_question_embeddings = []
for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    test_question = row['masked_question']
    question_embedding = get_embedding(
        test_question,
        engine="text-embedding-ada-002"
    )
    test_question_embeddings.append(question_embedding)

100%|██████████| 150/150 [00:22<00:00,  6.76it/s]


In [313]:
test['test_question_embedding'] = test_question_embeddings

In [314]:
#test = pd.read_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv')
#test["test_question_embedding"] = test.test_question_embedding.apply(eval).apply(np.array)

In [198]:
#test.to_csv('../data/QALD/9/data/qald-9-test.csv-with-embeddings-cot', index=None)

In [219]:
#train = pd.read_csv('../data/QALD/9/data/qald-9-train-with-embeddings-cot.csv')

In [220]:
#train["train_question_embedding"] = train.train_question_embedding.apply(eval).apply(np.array)

In [199]:
#train.to_csv('../data/QALD/9/data/qald-9-train-with-embeddings-cot.csv', index=None)

In [313]:
from openai.embeddings_utils import get_embedding, cosine_similarity

train_matched_questions = []
train_matched_queries = []
for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    test_question_embedding = row['test_question_embedding']
    
    train_embeddings_similarities = train_embeddings.question_embedding.apply(lambda x: cosine_similarity(x,test_question_embedding))
    
    max_idx = train_embeddings_similarities.idxmax()
    
    results = \
        train_embeddings.iloc[max_idx][['question', 'query']]
    
    matched_question = results['question']
    matched_query = results['query']

    train_matched_questions.append(matched_question)
    train_matched_queries.append(matched_query)

100%|██████████| 129/129 [00:24<00:00,  5.32it/s]


### Few-shot learning on matched train question and chain of thought

In [1214]:
from tqdm import tqdm

chatgpt_train_fewshot_query = []

count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    
    #if idx < 3:
    if (idx > -1) and (idx < 200):
    
        test_question = row['question_text']
        
        test_masked_question = row['masked_question']

        train_question_idx = train.index[train['question_text'] == test_question].tolist()

        if len(train_question_idx) > 0: # find a matched train question, skip to generate new query
            chatgpt_train_fewshot_query.append(train.iloc[train_question_idx[0]]['sparql_query'])

        else:
            
            test_masked_question_embedding = get_embedding(
                test_masked_question,
                engine="text-embedding-ada-002"
            )

            train_masked_embeddings_similarities = train.train_masked_question_embedding.\
            apply(lambda x: cosine_similarity(x,test_masked_question_embedding))

            top5_idx = train_masked_embeddings_similarities.sort_values(ascending=False)[:5].index

            top5_train_masked_questions = list(train.iloc[top5_idx].masked_question.values)
            top5_train_masked_queries = list(train.iloc[top5_idx].masked_query.values)
            top5_masked_cots = list(train.iloc[top5_idx].masked_cot.values)
            
            top5_train_questions = list(train.iloc[top5_idx].question_text.values)
            top5_train_queries = list(train.iloc[top5_idx].sparql_query.values)
            top5_cots = list(train.iloc[top5_idx].train_cot.values)
            
            msg = """
               Study the following example question, logical steps, 
               and SPARQL query provided:
           
               Question:{}
               Logical steps:{}
               SPARQL query: {}
           
               Now, using the learned pattern and logical steps, translate 
               the new question below into a SPARQL query.
               Write a syntactically corect SPARQL query only.
               The query should return answers as {}.
               Include all required prefixes in the query. 
               No comments. Output query only.
               
               New question: {}
               Query: 
            """
            msg = msg.format(top5_train_questions[0], top5_cots[0], \
                             top5_train_queries[0], answertype_text, test_question)

            #print(msg)
            
            response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role":"system", "content":"Reset your memory and start with a \
                 clean slate. Disregard any previous information or context from \
                 our conversation. "},
                {"role": "system", "content": "Now you are a helpful assistant focusing on DBpedia. \
                You translate a user question to a SPARQL query to answer the question \
                using the DBpedia knowledge base."},
                {"role": "user", "content": msg}
                ]
            )


            chatgpt_train_fewshot_query.append(response['choices'][0]['message']['content'].\
                                       strip().replace('\n', ' ' ))

100%|██████████| 150/150 [07:49<00:00,  3.13s/it]


In [1211]:
chatgpt_train_fewshot_query

['PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?uri WHERE { res:Salt_Lake_City dbo:timeZone ?uri }',
 'PREFIX dbo: <http://dbpedia.org/ontology/>                PREFIX prop: <http://dbpedia.org/property/>                                SELECT DISTINCT ?uri WHERE {                    ?uri a dbo:Person ;                         prop:murderedBy <http://dbpedia.org/resource/Julius_Caesar> . }',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> SELECT DISTINCT ?uri WHERE {      ?uri rdf:type dbo:Mountain ;          dbo:locatedInArea res:Germany ;          dbo:elevation ?elevation  } ORDER BY DESC(?elevation) LIMIT 1',
 'PREFIX dbo: <http://dbpedia.org/ontology/>                 PREFIX res: <http://dbpedia.org/resource/>                 SELECT DISTINCT ?president WHERE {                   ?president a dbo:President ;                         

In [1215]:
test['chatgpt_nomasked_train_cot_fewshot_query'] = chatgpt_train_fewshot_query

In [1217]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

### Retrieve chatgpt_train_cot_fewshot_query results

In [1218]:
# Retrieve the chatgpt_train_3fewshot_query_results
# query the DBpedia endpoint in
# March, 2023

from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import ast

# set up the SPARQL endpoint URL
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

sparql.setReturnFormat(JSON)

chatgpt_train_cot_fewshot_query_results = []
count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    #if count > 3:
    #    break
     
    count += 1
    chatgpt_train_fewshot_query = row['chatgpt_nomasked_train_cot_fewshot_query']
    
    sparql.setQuery(chatgpt_train_fewshot_query)
    
    try:
        ret = sparql.queryAndConvert()

        chatgpt_train_cot_fewshot_query_results.append(ret)
    except Exception as e:
        chatgpt_train_cot_fewshot_query_results.append(e)
        print(e)

  9%|▉         | 14/150 [00:37<05:18,  2.34s/it]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at \'California\' before \'.\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/>                 PREFIX dbp: <http://dbpedia.org/property/>                 PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>                 SELECT DISTINCT ?airport                 WHERE {                     ?airport a dbo:Airport .                     ?airport dbo:location ?location .                     ?location dbp:state California .                     ?location dbp:country "United States"@en .                     ?airport geo:lat ?lat .                     ?airport geo:long ?long .                 }\n'


 25%|██▍       | 37/150 [01:34<04:07,  2.19s/it]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at \'=\' before \'"en"\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX dbp: <http://dbpedia.org/property/> SELECT DISTINCT ?actor WHERE {   res:Lovesick dbo:starring ?uri .   ?uri dbo:starring ?actor .   ?uri dbo:language ?lang .   FILTER(lang = "en") }\n'


 36%|███▌      | 54/150 [02:09<02:56,  1.84s/it]

...

 85%|████████▌ | 128/150 [05:19<01:04,  2.92s/it]

EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Virtuoso 22023 Error SR066: Unsupported case in CONVERT (DATETIME -> INTEGER)\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/>                 PREFIX dbp: <http://dbpedia.org/property/>                 SELECT DISTINCT ?uri                  WHERE {                    ?uri a dbo:Library ;                    dbp:established ?year                    FILTER (xsd:integer(?year) < 1400)                  }\n'


 91%|█████████▏| 137/150 [05:41<00:29,  2.27s/it]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at ',' before '_Princess_of_Wales'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT DISTINCT ?uri WHERE {   res:Diana,_Princess_of_Wales dbo:dateOfDeath ?uri. }\n"


100%|██████████| 150/150 [06:24<00:00,  2.56s/it]


In [1219]:
test['chatgpt_nomasked_train_cot_fewshot_query_results'] = chatgpt_train_cot_fewshot_query_results

In [1220]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

### Few-shot learning on matched train question only

In [1235]:
test.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers', 'gpt_answers_text_DBpedia-2016-04',
       'chatgpt_answers_text_DBpedia_2016_04',
       'gold_query_results_DBpedia_2023_03',
       'chatgpt_answers_text_DBpedia_2023_03', 'gpt_query_DBpedia_2023_03',
       'gpt_query_results_DBpedia_2023_03', 'chatgpt_query_DBpedia_2023_03',
       'chatgpt_query_results_DBpedia_2023_03', 'test_question_embedding',
       'chatgpt_train_3fewshot_query', 'chatgpt_train_3fewshot_query_results',
       'chatgpt_train_1fewshot_query', 'chatgpt_train_1fewshot_query_results',
       'gpt_fewshot_query', 'gpt_fewshot_query_results', 'cot',
       'chatgpt_cot_query', 'chatgpt_cot_query_results', 'masked_question',
       'masked_query', 'masked_cot', 'chatgpt_train_cot_fewshot_query',
       'chatgpt_train_cot_fewshot_query_results',
       'chatgpt_nomasked_tra

In [1236]:
train.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers', 'train_question_embedding',
       'similarity', 'train_cot', 'masked_question', 'masked_query',
       'train_masked_question_embedding', 'masked_cot'],
      dtype='object')

In [1240]:
from tqdm import tqdm

chatgpt_train_fewshot_query = []

count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    
    #if idx < 3:
    if (idx > -1) and (idx < 200):
    
        test_question = row['question_text']
        
        test_masked_question = row['masked_question']

        train_question_idx = train.index[train['question_text'] == test_question].tolist()

        if len(train_question_idx) > 0: # find a matched train question, skip to generate new query
            chatgpt_train_fewshot_query.append(train.iloc[train_question_idx[0]]['sparql_query'])

        else:
            
            test_masked_question_embedding = get_embedding(
                test_masked_question,
                engine="text-embedding-ada-002"
            )

            train_masked_embeddings_similarities = train.train_masked_question_embedding.\
            apply(lambda x: cosine_similarity(x,test_masked_question_embedding))

            top5_idx = train_masked_embeddings_similarities.sort_values(ascending=False)[:5].index

            top5_train_masked_questions = list(train.iloc[top5_idx].masked_question.values)
            top5_train_masked_queries = list(train.iloc[top5_idx].masked_query.values)
            top5_masked_cots = list(train.iloc[top5_idx].masked_cot.values)
            
            top5_train_questions = list(train.iloc[top5_idx].question_text.values)
            top5_train_queries = list(train.iloc[top5_idx].sparql_query.values)
            top5_cots = list(train.iloc[top5_idx].train_cot.values)
            
            msg = """
               Study the following example question and SPARQL query provided:
           
               Question:{}
               SPARQL query: {}
           
               Now, using the learned pattern to translate 
               the new question below into a SPARQL query.
               Write a syntactically corect SPARQL query only.
               The query should return answers as {}.
               Include all required prefixes in the query. 
               No comments. Output query only.
               
               New question: {}
               Query: 
            """
            msg = msg.format(top5_train_questions[0], \
                             top5_train_queries[0], answertype_text, test_question)

            #print(msg)
            
            response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role":"system", "content":"Reset your memory and start with a \
                 clean slate. Disregard any previous information or context from \
                 our conversation. "},
                {"role": "system", "content": "Now you are a helpful assistant focusing on DBpedia. \
                You translate a user question to a SPARQL query to answer the question \
                using the DBpedia knowledge base."},
                {"role": "user", "content": msg}
                ]
            )


            chatgpt_train_fewshot_query.append(response['choices'][0]['message']['content'].\
                                       strip().replace('\n', ' ' ))

100%|██████████| 150/150 [08:45<00:00,  3.50s/it]


In [1246]:
chatgpt_train_fewshot_query

['PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?uri WHERE { res:Salt_Lake_City dbo:timeZone ?uri }',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbp: <http://dbpedia.org/property/> SELECT DISTINCT ?uri WHERE {   ?uri a dbo:Person ;        dbp:subject <http://dbpedia.org/resource/Assassination_of_Julius_Caesar> . }',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> SELECT DISTINCT ?uri WHERE {    ?uri rdf:type dbo:Mountain ;         dbo:locatedInArea res:Germany ;         dbo:elevation ?num  }  ORDER BY DESC(?num) OFFSET 0 LIMIT 1',
 'PREFIX dbpedia: <http://dbpedia.org/resource/> PREFIX dbpprop: <http://dbpedia.org/property/> SELECT DISTINCT ?president WHERE {   ?president dbpprop:office dbpedia:President_of_the_United_States ;              dbpprop:termPeriod dbpedia:Vietnam_War ;              dbpprop:succeededBy ?successor . 

In [1213]:
len(chatgpt_train_fewshot_query)

142

In [1248]:
test['chatgpt_nomasked_train_only_fewshot_query'] = chatgpt_train_fewshot_query

In [1217]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

### Retrieve chatgpt_train_only_fewshot_query results

In [1249]:
# Retrieve the chatgpt_train_3fewshot_query_results
# query the DBpedia endpoint in
# March, 2023

from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import ast

# set up the SPARQL endpoint URL
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

sparql.setReturnFormat(JSON)

chatgpt_train_cot_fewshot_query_results = []
count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    #if count > 3:
    #    break
     
    count += 1
    chatgpt_train_fewshot_query = row['chatgpt_nomasked_train_only_fewshot_query']
    
    sparql.setQuery(chatgpt_train_fewshot_query)
    
    try:
        ret = sparql.queryAndConvert()

        chatgpt_train_cot_fewshot_query_results.append(ret)
    except Exception as e:
        chatgpt_train_cot_fewshot_query_results.append(e)
        print(e)

  5%|▍         | 7/150 [00:06<01:58,  1.21it/s]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP031: SPARQL compiler: Internal error: sparp_expn_native_valmode(): unsupported case\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbpedia: <http://dbpedia.org/resource/> PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>  SELECT DISTINCT ?artist WHERE {?artist a dbpedia-owl:Artist . ?artist dbpedia-owl:birthDate ?date . FILTER(?date = (SELECT DISTINCT ?birthdate WHERE {dbpedia:Rachel_Stevens dbpedia-owl:birthDate ?birthdate})) . FILTER(?artist != dbpedia:Rachel_Stevens)}\n'


 37%|███▋      | 56/150 [00:55<02:21,  1.50s/it]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at '<http://dbpedia.org/resource/Veganism>' before '}'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX foaf: <http://xmlns.com/foaf/0.1/> ASK WHERE { res:Pamela_Anderson dbo:diet rdf:resource <http://dbpedia.org/resource/Veganism> }\n"


 55%|█████▌    | 83/150 [01:21<01:08,  1.02s/it]

...

 91%|█████████▏| 137/150 [02:10<00:08,  1.52it/s]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at ',' before '_Princess_of_Wales'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT DISTINCT ?uri WHERE { res:Diana,_Princess_of_Wales dbo:deathDate ?uri. }\n"


 92%|█████████▏| 138/150 [02:11<00:07,  1.55it/s]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 3: syntax error at '}'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbp: <http://dbpedia.org/property/> SELECT DISTINCT ?uri WHERE {     <http://dbpedia.org/resource/Intel> dbo:fo#efounder ?uri . }\n"


100%|██████████| 150/150 [02:23<00:00,  1.05it/s]


In [1250]:
test['chatgpt_nomasked_train_only_fewshot_query_results'] = chatgpt_train_cot_fewshot_query_results

In [1251]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

### Few-shot learning on matched train question 3-shot only

In [1274]:
from tqdm import tqdm

chatgpt_train_fewshot_query = []

count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    
    #if idx < 3:
    if (idx > 51) and (idx < 200):
    
        test_question = row['question_text']
        
        test_masked_question = row['masked_question']

        train_question_idx = train.index[train['question_text'] == test_question].tolist()

        if len(train_question_idx) > 0: # find a matched train question, skip to generate new query
            chatgpt_train_fewshot_query.append(train.iloc[train_question_idx[0]]['sparql_query'])

        else:
            
            test_masked_question_embedding = get_embedding(
                test_masked_question,
                engine="text-embedding-ada-002"
            )

            train_masked_embeddings_similarities = train.train_masked_question_embedding.\
            apply(lambda x: cosine_similarity(x,test_masked_question_embedding))

            top5_idx = train_masked_embeddings_similarities.sort_values(ascending=False)[:5].index

            top5_train_masked_questions = list(train.iloc[top5_idx].masked_question.values)
            top5_train_masked_queries = list(train.iloc[top5_idx].masked_query.values)
            top5_masked_cots = list(train.iloc[top5_idx].masked_cot.values)
            
            top5_train_questions = list(train.iloc[top5_idx].question_text.values)
            top5_train_queries = list(train.iloc[top5_idx].sparql_query.values)
            top5_cots = list(train.iloc[top5_idx].train_cot.values)
            
            msg = """
               Study the following example questions and SPARQL queries provided:
           
               Question:{}
               SPARQL query: {}
               
               Question:{}
               SPARQL query: {}
               
               Question:{}
               SPARQL query: {}
           
               Now, using the learned patterns to translate 
               the new question below into a SPARQL query.
               Write a syntactically corect SPARQL query only.
               The query should return answers as {}.
               Include all required prefixes in the query. 
               No comments. Output query only.
               
               New question: {}
               Query: 
            """
            msg = msg.format(top5_train_questions[0], top5_train_queries[0], \
                             top5_train_questions[1], top5_train_queries[1], \
                             top5_train_questions[2], top5_train_queries[2], \
                             answertype_text, test_question)

            #print(msg)
            
            response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role":"system", "content":"Reset your memory and start with a \
                 clean slate. Disregard any previous information or context from \
                 our conversation. "},
                {"role": "system", "content": "Now you are a helpful assistant focusing on DBpedia. \
                You translate a user question to a SPARQL query to answer the question \
                using the DBpedia knowledge base."},
                {"role": "user", "content": msg}
                ]
            )


            chatgpt_train_fewshot_query.append(response['choices'][0]['message']['content'].\
                                       strip().replace('\n', ' ' ))

100%|██████████| 150/150 [04:06<00:00,  1.64s/it]


In [1271]:
chatgpt_train_fewshot_query

['PREFIX dbo: <http://dbpedia.org/ontology/>                 PREFIX res: <http://dbpedia.org/resource/>                 SELECT DISTINCT ?uri WHERE {                     res:Salt_Lake_City dbo:timeZone ?uri                 }',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?uri WHERE {  res:Julius_Caesar dbo:causeOfDeath ?uri .  FILTER(CONTAINS(STR(?uri), "assassin"))  }',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> SELECT DISTINCT ?uri WHERE {       ?uri rdf:type dbo:Mountain ;      dbo:locatedInArea res:Germany ;      dbo:elevation ?elevation  }  ORDER BY DESC(?elevation)  LIMIT 1',
 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?uri WHERE {     ?uri dbo:office <http://dbpedia.org/resource/President_of_the_United_States> ;          dbo:position <http://dbpedia.org/resource

In [1278]:
#test['chatgpt_nomasked_train_only_3fewshot_query'] = chatgpt_train_fewshot_query
test['chatgpt_nomasked_train_only_3fewshot_query'] = temp_query_52

In [1279]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

### Retrieve chatgpt_train_only_3fewshot_query results

In [1280]:
# Retrieve the chatgpt_train_3fewshot_query_results
# query the DBpedia endpoint in
# March, 2023

from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import ast

# set up the SPARQL endpoint URL
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

sparql.setReturnFormat(JSON)

chatgpt_train_cot_fewshot_query_results = []
count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    #if count > 3:
    #    break
     
    count += 1
    chatgpt_train_fewshot_query = row['chatgpt_nomasked_train_only_3fewshot_query']
    
    sparql.setQuery(chatgpt_train_fewshot_query)
    
    try:
        ret = sparql.queryAndConvert()

        chatgpt_train_cot_fewshot_query_results.append(ret)
    except Exception as e:
        chatgpt_train_cot_fewshot_query_results.append(e)
        print(e)

  3%|▎         | 4/150 [00:08<04:40,  1.92s/it]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at '?vc' before '.'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?uri WHERE {     ?uri dbo:office <http://dbpedia.org/resource/President_of_the_United_States> ;          dbo:position <http://dbpedia.org/resource/President_of_the_United_States> ;          dbo:officeholding/res:Vietnam_War dbo:militaryCommander ?vc . }\n"


 19%|█▊        | 28/150 [00:39<02:37,  1.29s/it]

EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Virtuoso 22023 Error SL001: The SPARQL 1.1 function STRSTARTS() needs a string value as first argument\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX foaf: <http://xmlns.com/foaf/0.1/> PREFIX res: <http://dbpedia.org/resource/>  SELECT DISTINCT ?homepage WHERE {     res:Forbes dbo:wikiPageExternalLink ?homepage .     FILTER(strStarts(?homepage, "http://www.forbes.com/"))  }\n'


 29%|██▊       | 43/150 [00:55<01:47,  1.01s/it]

...

 69%|██████▉   | 104/150 [01:44<00:30,  1.52it/s]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at 'dbo:Earl' before ';'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?person WHERE {   ?person rdf:type dbo:Noble ,                        dbo:Person ;           dbo:deathPlace ?place ;           dbo:birthPlace ?place ;           dbo:father ?father ;           ?father rdf:type dbo:Earl ;           ?father dbo:Nationality res:United_Kingdom . }\n"


 90%|█████████ | 135/150 [02:17<00:21,  1.41s/it]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at 'group_or_band' before ';'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?studio WHERE {     res:The_Beatles dbo:background group_or_band ;                     dbo:recordedIn ?studio .     ?album rdf:type dbo:Album ;                   dbo:artist res:The_Beatles ;                   dbo:recordLabel ?label ;                   dbo:recordedIn ?studio . } ORDER BY ASC(?album) OFFSET 0 LIMIT 1\n"


100%|██████████| 150/150 [02:27<00:00,  1.02it/s]


In [1281]:
test['chatgpt_nomasked_train_only_3fewshot_query_results'] = chatgpt_train_cot_fewshot_query_results

In [1282]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

### Evaluate the chatgpt_train_only_3fewshot_query_results

In [101]:
import json, ast

In [391]:
test.shape

(150, 26)

In [1182]:
# retrieve query results
import ast

query_results_terms = []
count = 0
for idx, row in test.iterrows():
    try:
        bindings = ast.literal_eval(row['gold_query_results_DBpedia_2023_03'])['results']['bindings']

        answer_list = []
        for item in bindings:
            for k in item:
                answer_list.append(item[k]['value'])

        terms = []
        for ans in answer_list:
            terms.append(ans.replace('http://dbpedia.org/resource/', '').replace('dbo:', '').strip().lower())
        #if terms not in answer_terms:
        query_results_terms.append(terms)
              
    except:
        print(row['gold_query_results_DBpedia_2023_03'])
        ex_ans = ast.literal_eval(row['gold_query_results_DBpedia_2023_03'])['boolean']
        if ex_ans:
            query_results_terms.append([str(ex_ans).lower()])
        else:
            query_results_terms.append([])
        count += 1

{'head': {'link': []}, 'boolean': True}
{'head': {'link': []}, 'boolean': True}
{'head': {'link': []}, 'boolean': True}
{'head': {'link': []}, 'boolean': True}


In [393]:
len(query_results_terms)

150

In [39]:
query_results_terms

[['mountain_time_zone'],
 ['decimus_junius_brutus_albinus',
  'gaius_cassius_longinus',
  'marcus_junius_brutus',
  'pontius_aquila',
  'pacuvius_labeo',
  'quintus_ligarius',
  'publius_servilius_casca',
  'gaius_cassius_parmensis',
  'gaius_trebonius',
  'lucius_minucius_basilus',
  'marcus_porcius_cato_(son_of_cato_the_younger)',
  'tillius_cimber'],
 ['zugspitze'],
 ['richard_nixon', 'lyndon_b._johnson'],
 [],
 ['8'],
 ['vesna_pisarović', 'gizem_saka', 'kelly_kelekidou', 'cameron_cartio'],
 ['novelist'],
 ['true'],
 ['50035'],
 ['miles_&_more'],
 ['andorra', 'denmark', 'belgium', 'united_kingdom', 'sweden'],
 ['georgia_(country)',
  'western_australia',
  'jamaica',
  'japan',
  'northern_territory',
  'spain',
  'india',
  'venezuela',
  'derbyshire',
  'united_states',
  'turkey',
  'somerset',
  'france',
  'germany',
  'canada',
  'austria',
  'republic_of_ireland',
  'italy',
  'cumbria',
  'taiwan',
  'brazil',
  'north_yorkshire',
  'gozo',
  'philippines',
  'mexico',
  'vi

In [394]:
test.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers', 'gpt_answers_text_DBpedia-2016-04',
       'chatgpt_answers_text_DBpedia_2016_04',
       'gold_query_results_DBpedia_2023_03',
       'chatgpt_answers_text_DBpedia_2023_03', 'gpt_query_DBpedia_2023_03',
       'gpt_query_results_DBpedia_2023_03', 'chatgpt_query_DBpedia_2023_03',
       'chatgpt_query_results_DBpedia_2023_03', 'test_question_embedding',
       'chatgpt_train_3fewshot_query', 'chatgpt_train_3fewshot_query_results',
       'chatgpt_train_1fewshot_query', 'chatgpt_train_1fewshot_query_results'],
      dtype='object')

In [1284]:
# retrieve chatgpt train 3fewshot query results
import ast

chatgpt_query_results_terms = []
count = 0
for idx, row in test.iterrows():
    try:
        bindings = row['chatgpt_nomasked_train_only_3fewshot_query_results']['results']['bindings']
        #bindings = ast.literal_eval(row['chatgpt_train_1fewshot_query_results'])['results']['bindings']

        answer_list = []
        for item in bindings:
            for k in item:
                answer_list.append(item[k]['value'])

        terms = []
        for ans in answer_list:
            terms.append(ans.replace('http://dbpedia.org/resource/', '').replace('dbo:', '').strip().lower())
        #if terms not in answer_terms:
        chatgpt_query_results_terms.append(terms)
    except TypeError:
        chatgpt_query_results_terms.append(['ERROR ERROR ERROR'])
    except SyntaxError:
        chatgpt_query_results_terms.append(['ERROR ERROR ERROR'])
              
    except:
        print(row['chatgpt_nomasked_train_only_3fewshot_query_results'])
        ex_ans = row['chatgpt_nomasked_train_only_3fewshot_query_results']['boolean']
        #ex_ans = ast.literal_eval(row['chatgpt_train_1fewshot_query_results'])['boolean']
        if ex_ans:
            chatgpt_query_results_terms.append([str(ex_ans).lower()])
        else:
            chatgpt_query_results_terms.append([])
        count += 1

{'head': {'link': []}, 'boolean': False}


In [1285]:
len(chatgpt_query_results_terms)

150

In [550]:
chatgpt_query_results_terms

[['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR ERROR'],
 ['ERROR ERROR E

In [1286]:
# Evaluate the precision and recall based on the total numbers of 
# gold answers and predicted answers
predicted = 0
gold = 0
predicted_correct = 0
some_matched = {}
pre_gold_lengths = []
for idx, pred_terms in enumerate(chatgpt_query_results_terms):
    
    gold_terms = query_results_terms[idx]
    
    predicted +=  len(pred_terms)
    gold += len(gold_terms)
    
    pre_gold_lengths.append((idx, len(pred_terms), len(gold_terms)))
    
    if (len(pred_terms) > 0) and (len(gold_terms) > 0):
        predicted_correct_idx = False
        for pterm in pred_terms:
            if len(pterm) > 0: # skip an empty string
                for gterm in gold_terms:
                    if len(gterm) > 0:
                        #if pterm ==  gterm:
                        pterm = pterm.replace("_", " ")
                        gterm = gterm.replace("_", " ")
                        if (pterm in gterm) or (gterm in pterm):
                            predicted_correct_idx = True
                            predicted_correct += 1
                            #break # this pterm is a correct prediction, skip to next pterm
                                    # don't double count this pterm anymore

        some_matched[idx] = predicted_correct_idx

In [1287]:
pre_gold_lengths

[(0, 1, 1),
 (1, 0, 12),
 (2, 1, 1),
 (3, 1, 2),
 (4, 0, 0),
 (5, 1, 1),
 (6, 4, 4),
 (7, 0, 1),
 (8, 0, 1),
 (9, 1, 1),
 (10, 2, 1),
 (11, 0, 5),
 (12, 48, 48),
 (13, 0, 85),
 (14, 0, 1),
 (15, 1, 1),
 (16, 0, 0),
 (17, 0, 61),
 (18, 0, 1),
 (19, 0, 0),
 (20, 1, 1),
 (21, 0, 0),
 (22, 0, 0),
 (23, 240, 1714),
 (24, 1, 1),
 (25, 0, 1),
 (26, 5, 1),
 (27, 1, 1),
 (28, 0, 0),
 (29, 1, 1),
 (30, 1, 1),
 (31, 0, 0),
 (32, 0, 10),
 (33, 0, 0),
 (34, 0, 1),
 (35, 26, 26),
 (36, 0, 2),
 (37, 0, 1),
 (38, 9, 3),
 (39, 0, 0),
 (40, 1, 1),
 (41, 1, 1),
 (42, 1, 1),
 (43, 2, 0),
 (44, 26, 26),
 (45, 0, 1),
 (46, 0, 1),
 (47, 0, 0),
 (48, 1, 1),
 (49, 0, 0),
 (50, 0, 1),
 (51, 0, 1),
 (52, 0, 197),
 (53, 1, 1),
 (54, 1, 1),
 (55, 1, 1),
 (56, 0, 15),
 (57, 2, 2),
 (58, 0, 4),
 (59, 0, 0),
 (60, 0, 0),
 (61, 1, 1),
 (62, 0, 0),
 (63, 1, 1),
 (64, 0, 0),
 (65, 76, 164),
 (66, 1, 1),
 (67, 0, 1),
 (68, 1, 1),
 (69, 0, 36),
 (70, 0, 10),
 (71, 74, 74),
 (72, 8, 8),
 (73, 0, 20),
 (74, 43, 44),
 (75, 0

In [694]:
some_matched[0]

True

In [1288]:
precision = predicted_correct / (predicted)
precision

0.8111455108359134

In [1289]:
recall = predicted_correct/gold
recall

0.21510673234811165

In [1290]:
f1 = 2 / (1/precision + 1/recall)
f1

0.34003893575600264

In [1291]:
adj_precision = (predicted_correct-240) / (predicted - 240)
adj_recall = (predicted_correct-240) / (gold - 1714)
adj_f1 = 2 / (1/adj_precision +  1/adj_recall)
print('adj_precision:{},\nadj_recall:{},\nadj_f1:{}'.format(adj_precision, adj_recall, adj_f1))
predicted_correct, predicted, gold

adj_precision:0.7489711934156379,
adj_recall:0.28144329896907216,
adj_f1:0.40914200074934437


(786, 969, 3654)

In [1292]:
# Evaluate the precision and recall based on the total numbers of test questions
count = 0
predicted_correct = 0
some_matched = {}
for idx, pred_terms in enumerate(chatgpt_query_results_terms):
    gold_terms = query_results_terms[idx]
    
    count += 1
    
    if (len(pred_terms) > 0) and (len(gold_terms) > 0):
        predicted_correct_idx = False
        for pterm in pred_terms:
            if len(pterm) > 0: # skip an empty string
                for gterm in gold_terms:
                    if len(gterm) > 0:
                        #if pterm ==  gterm:
                        #pterm = pterm.replace("_", " ")
                        #gterm = gterm.replace("_", " ")
                        if not predicted_correct_idx:
                            #if (pterm in gterm) or (gterm in pterm):
                            if pterm == gterm:
                                predicted_correct_idx = True
                                predicted_correct += 1
                        else:
                            pass
                
        some_matched[idx] = predicted_correct_idx
    elif (len(pred_terms) == 0) and (len(gold_terms) == 0):
        predicted_correct += 1
        some_matched[idx] = True

In [1293]:
total = 0
for k in some_matched:
    if some_matched[k]:
        total += 1
total

75

In [1294]:
count

150

In [1295]:
predicted_correct

75

In [1296]:
precision = predicted_correct / count
precision

0.5

In [1297]:
recall = predicted_correct/count
recall

0.5

In [1298]:
f1 = 2 / (1/precision + 1/recall)
f1

0.5

### Explain test query in chain of thought without word limit

In [1306]:
from tqdm import tqdm

cot = []

count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    
    #if count > 4:
    #    break
    count += 1
    
    test_query = row['sparql_query']
    
    #if idx < 3:
    if (idx > 123) and (idx < 200):
        msg = """
               Briefly explain the following query in logical steps as a chain of thought. 
               Explain in natural language. 
               Forget what you have about the query before. 
               Assume you are trying to construct the query again. 
               No comments. Output the steps only. 
               Do not include the original query in the explanation. 

               QUERY:{}
               THOUGHT:
        """
        msg = msg.format(test_query)

        response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "you are a helpful assistant focusing on DBpedia."},
                {"role": "user", "content": msg}
                ]
        )


        cot.append(response['choices'][0]['message']['content'].\
                                       strip().replace('\n', ' ' ))

100%|██████████| 150/150 [01:47<00:00,  1.40it/s]


In [1307]:
cot

['1. Retrieve all triples where the predicate is "a" and the object is dbo:MilitaryConflict. 2. Retrieve all triples where the predicate is dbo:place and the object is dbr:San_Antonio. 3. Retrieve all triples where the predicate is dbo:date. 4. Filter the results by only keeping triples where the date is greater than December 31, 1835. 5. Filter the results again by only keeping triples where the date is less than or equal to December 31, 1836. 6. Select the subject of the remaining triples as the output (?uri).',
 '1. Define the namespace prefixes for the RDF schema, RDF syntax, Friend of a Friend, and DBpedia ontology. 2. Select distinct URIs. 3. Define that the URI must have a relationship with the DBpedia resource "Charmed". 4. The relationship must be that the selected URI is listed as a starring actor in the resource "Charmed". 5. Define that the selected URI must have a relationship with a Friend of a Friend homepage. 6. Output the final list of distinct URIs meeting all of the 

In [1310]:
test['cot_noWordLimit']=temp_cot_124

In [1311]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

### ChatGPT get test query based on ChainOfThought

In [1320]:
from tqdm import tqdm

chatgpt_cot_query = []

count = 0

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    
    #if count > 4:
    #    break
    count += 1
    
    test_question = row['question_text']

    cot = row['cot_noWordLimit']
    
    answertype = row['answertype']
    answertype_text = ""
    if answertype == 'resource':
        answertype_text = 'DBpedia Resource URI(s)'
    else:
        answertype_text = answertype
    
    #if idx < 3:
    if (idx > 120) and (idx < 200):
        msg = """
               Translate the question to SPARQL query on DBpedia. 
               Forget what you have about the question and query before. 
               Follow the steps in the chain of thought to construct the query. 
               Output query only. No comments. 
               Include all required prefixes in the query. 
               Return the answers as {}.

               QUESTION: {}

               THOUGHT: {}

               QUERY: 
        """
        msg = msg.format(answertype_text, test_question, cot)

        response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "you are a helpful assistant focusing on DBpedia. \
                You will translate a user question to a SPARQL query on the DBpedia knowledge base."},
                {"role": "user", "content": msg}
                ]
        )


        chatgpt_cot_query.append(response['choices'][0]['message']['content'].\
                                       strip().replace('\n', ' ' ))

100%|██████████| 150/150 [01:23<00:00,  1.80it/s]


In [1325]:
test['chatgpt_cot_noWordLimit_query'] = temp_cot_query_121

In [1326]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

### Retrieve chatgpt_cot_noWordLimit_query results

In [1328]:
# Retrieve the chatgpt_cot_query_results

from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import ast

# set up the SPARQL endpoint URL
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

sparql.setReturnFormat(JSON)

chatgpt_cot_query_results = []
count = 0
for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    #if count > 3:
    #    break 
    count += 1
    
    gpt_query = row['chatgpt_cot_noWordLimit_query']
    
    sparql.setQuery(gpt_query)
    
    try:
        ret = sparql.queryAndConvert()

        chatgpt_cot_query_results.append(ret)
    except Exception as e:
        chatgpt_cot_query_results.append(e)
        print(e)

 29%|██▉       | 44/150 [00:39<02:02,  1.16s/it]

EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Virtuoso 22003 Error SR087: Non numeric argument(s) to arithmetic operation \'/\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX dbr: <http://dbpedia.org/resource/>  SELECT DISTINCT ?surfer WHERE {   ?surfer a dbo:Person ;           dbo:occupation dbr:Surfing ;           dbo:birthPlace ?birthPlace .      FILTER (regex(?birthPlace, "Australia", "i") || regex(?birthPlace/dbo:country, "Australia", "i")) }\n'


 45%|████▍     | 67/150 [00:57<00:53,  1.54it/s]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at 'isDate' before '('\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX foaf: <http://xmlns.com/foaf/0.1/> SELECT DISTINCT ?deathDate  WHERE {   <http://dbpedia.org/resource/Michael_Jackson> dbo:deathDate ?deathDate .   FILTER(isDate(?deathDate)) }\n"


 49%|████▊     | 73/150 [01:02<01:35,  1.24s/it]

...

 85%|████████▌ | 128/150 [02:03<00:39,  1.80s/it]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at '<' before '1400'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbpedia: <http://dbpedia.org/resource/> PREFIX dbo: <http://dbpedia.org/ontology/>  SELECT DISTINCT ?library WHERE {   ?library a dbo:Library ;            dbo:established ?date .   FILTER (xsd:integer(substr(str(?date), 1, 4))) < 1400 . }\n"


 97%|█████████▋| 146/150 [02:17<00:02,  1.44it/s]

QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at \'FILTER\' before \'EXISTS\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nPREFIX dbo: <http://dbpedia.org/ontology/> PREFIX geo: <http://www.opengis.net/ont/geosparql#> SELECT DISTINCT ?trail WHERE {   ?trail a dbo:HikingTrail ;          dbo:trailHead <http://dbpedia.org/resource/Grand_Canyon_National_Park> ;          FILTER NOT EXISTS { ?trail dbo:hazards "Flash flood" } . }\n'


100%|██████████| 150/150 [02:20<00:00,  1.07it/s]


In [1330]:
test['chatgpt_cot_noWordLimit_query_results'] = chatgpt_cot_query_results

In [1331]:
#test.to_csv('../data/QALD/9/data/qald-9-test-with-embeddings-cot.csv', index=None)

### Evaluate the chatgpt_cot_noWordLimit_query_results

In [1335]:
# retrieve query results
import ast

query_results_terms = []
count = 0
for idx, row in test.iterrows():
    try:
        bindings = ast.literal_eval(row['gold_query_results_DBpedia_2023_03'])['results']['bindings']

        answer_list = []
        for item in bindings:
            for k in item:
                answer_list.append(item[k]['value'])

        terms = []
        for ans in answer_list:
            terms.append(ans.replace('http://dbpedia.org/resource/', '').replace('dbo:', '').strip().lower())
        #if terms not in answer_terms:
        query_results_terms.append(terms)
              
    except:
        print(row['gold_query_results_DBpedia_2023_03'])
        ex_ans = ast.literal_eval(row['gold_query_results_DBpedia_2023_03'])['boolean']
        if ex_ans:
            query_results_terms.append([str(ex_ans).lower()])
        else:
            query_results_terms.append([])
        count += 1

{'head': {'link': []}, 'boolean': True}
{'head': {'link': []}, 'boolean': True}
{'head': {'link': []}, 'boolean': True}
{'head': {'link': []}, 'boolean': True}


In [597]:
len(query_results_terms)

150

In [1336]:
# retrieve gpt query results
import ast

chatgpt_cot_query_results_terms = []
count = 0
for idx, row in test.iterrows():
    try:
        #bindings = ast.literal_eval(row['gpt_fewshot_query_results'])['results']['bindings']
        bindings = row['chatgpt_cot_noWordLimit_query_results']['results']['bindings']

        answer_list = []
        for item in bindings:
            for k in item:
                answer_list.append(item[k]['value'])

        terms = []
        for ans in answer_list:
            terms.append(ans.replace('http://dbpedia.org/resource/', '').replace('dbo:', '').strip().lower())
        #if terms not in answer_terms:
        chatgpt_cot_query_results_terms.append(terms)
    except SyntaxError:
        chatgpt_cot_query_results_terms.append(['ERROR ERROR ERROR'])
        
    except TypeError:
        chatgpt_cot_query_results_terms.append(['ERROR ERROR ERROR'])
              
    except:
        print(row['chatgpt_cot_noWordLimit_query_results'])
        #ex_ans = ast.literal_eval(row['gpt_query_fewshot_results_DBpedia'])['boolean']
        ex_ans = row['chatgpt_cot_noWordLimit_query_results']['boolean']
        if ex_ans:
            chatgpt_cot_query_results_terms.append([str(ex_ans).lower()])
        else:
            chatgpt_cot_query_results_terms.append([])
        count += 1

{'head': {'link': []}, 'boolean': True}
{'head': {'link': []}, 'boolean': True}
{'head': {'link': []}, 'boolean': True}
{'head': {'link': []}, 'boolean': True}


In [1337]:
len(chatgpt_cot_query_results_terms)

150

In [1355]:
# Evaluate the precision and recall based on the total numbers of 
# gold answers and predicted answers
predicted = 0
gold = 0
predicted_correct = 0
some_matched = {}

pred_gold_lengths = []
for idx, pred_terms in enumerate(chatgpt_cot_query_results_terms):
    
    gold_terms = query_results_terms[idx]
    
    predicted +=  len(pred_terms)
    gold += len(gold_terms)
    
    pred_gold_lengths.append((idx, len(pred_terms), len(gold_terms)))
    
    if (len(pred_terms) > 0) and (len(gold_terms) > 0):

        predicted_correct_idx = False
        for pterm in pred_terms:
            if len(pterm) > 0: # skip an empty string
                for gterm in gold_terms:
                    if len(gterm) > 0:
                        if pterm ==  gterm:
                        #pterm = pterm.replace("_", " ")
                        #gterm = gterm.replace("_", " ")
                        #if (pterm in gterm) or (gterm in pterm):
                            predicted_correct_idx = True
                            predicted_correct += 1
                            break # this pterm is a correct prediction, skip to next pterm
                                    # don't double count this pterm anymore

        some_matched[idx] = predicted_correct_idx

In [1344]:
pred_gold_lengths

[(0, 1, 1),
 (1, 0, 12),
 (2, 1, 1),
 (3, 2, 2),
 (4, 0, 0),
 (5, 1, 1),
 (6, 0, 4),
 (7, 1, 1),
 (8, 1, 1),
 (9, 1, 1),
 (10, 1, 1),
 (11, 5, 5),
 (12, 0, 48),
 (13, 82, 85),
 (14, 1, 1),
 (15, 1, 1),
 (16, 0, 0),
 (17, 19, 61),
 (18, 0, 1),
 (19, 0, 0),
 (20, 1, 1),
 (21, 0, 0),
 (22, 0, 0),
 (23, 240, 1714),
 (24, 0, 1),
 (25, 0, 1),
 (26, 0, 1),
 (27, 1, 1),
 (28, 0, 0),
 (29, 1, 1),
 (30, 1, 1),
 (31, 0, 0),
 (32, 10, 10),
 (33, 0, 0),
 (34, 1, 1),
 (35, 26, 26),
 (36, 0, 2),
 (37, 1, 1),
 (38, 0, 3),
 (39, 0, 0),
 (40, 1, 1),
 (41, 1, 1),
 (42, 1, 1),
 (43, 1, 0),
 (44, 2, 26),
 (45, 1, 1),
 (46, 1, 1),
 (47, 0, 0),
 (48, 1, 1),
 (49, 0, 0),
 (50, 0, 1),
 (51, 0, 1),
 (52, 197, 197),
 (53, 2, 1),
 (54, 1, 1),
 (55, 1, 1),
 (56, 15, 15),
 (57, 0, 2),
 (58, 4, 4),
 (59, 0, 0),
 (60, 0, 0),
 (61, 1, 1),
 (62, 0, 0),
 (63, 1, 1),
 (64, 0, 0),
 (65, 4, 164),
 (66, 1, 1),
 (67, 0, 1),
 (68, 0, 1),
 (69, 0, 36),
 (70, 37, 10),
 (71, 74, 74),
 (72, 1, 8),
 (73, 0, 20),
 (74, 10000, 44),


In [1343]:
pres = 0
gols = 0
for _, pre, gol in pred_gold_lengths:
    pres += pre
    gols += gol
pres, gols

(11452, 3654)

In [1339]:
precision = (predicted_correct) / (predicted)
precision

0.12198742577715684

In [1340]:
recall = predicted_correct/gold
recall

0.3823207443897099

In [1341]:
f1 = 2 / (1/precision + 1/recall)
f1

0.18495961869455843

In [1356]:
adj_precision = (predicted_correct - 240) / (predicted - 10000 -1)
adj_recall = (predicted_correct - 240) / (gold - 44 - 1714)
adj_f1 = 2 / (1/adj_precision +  1/adj_recall)
print('adj_precision:{},\nadj_recall:{},\nadj_f1:{}'.format(adj_precision, adj_recall, adj_f1))
predicted_correct, predicted, gold

adj_precision:0.7973811164713991,
adj_recall:0.6102320675105485,
adj_f1:0.6913654018524051


(1397, 11452, 3654)

In [1347]:
# Evaluate the precision and recall based on the total numbers of test questions
count = 0
predicted_correct = 0
some_matched = {}
for idx, pred_terms in enumerate(chatgpt_cot_query_results_terms):
    gold_terms = query_results_terms[idx]
    
    count += 1
    
    if (len(pred_terms) > 0) and (len(gold_terms) > 0):
        predicted_correct_idx = False
        for pterm in pred_terms:
            if len(pterm) > 0: # skip an empty string
                for gterm in gold_terms:
                    if len(gterm) > 0:
                        pterm = pterm.replace("_", " ")
                        gterm = gterm.replace("_", " ")
                        if not predicted_correct_idx:
                            if (pterm in gterm) or (gterm in pterm):
                                predicted_correct_idx = True
                                predicted_correct += 1
                        else:
                            pass
                
        some_matched[idx] = predicted_correct_idx
    elif (len(pred_terms) == 0) and (len(gold_terms) == 0):
        predicted_correct += 1
        some_matched[idx] = True

In [1348]:
total = 0
for k in some_matched:
    if some_matched[k]:
        total += 1
total

109

In [1349]:
count

150

In [1350]:
predicted_correct

109

In [1351]:
precision = predicted_correct / count
precision

0.7266666666666667

In [1352]:
recall = predicted_correct/count
recall

0.7266666666666667

In [1353]:
f1 = 2 / (1/precision + 1/recall)
f1

0.7266666666666667

### Explain train query in chain of thought and few-shot learning

In [744]:
train.columns

Index(['id', 'answertype', 'aggregation', 'onlydbo', 'hybrid', 'question_text',
       'question_keywords', 'sparql_query', 'answer_head', 'answer_results',
       'question', 'query', 'answers', 'train_question_embedding',
       'similarity'],
      dtype='object')

In [760]:
from tqdm import tqdm

train_cot = []

count = 0

for idx, row in tqdm(train.iterrows(), total=train.shape[0]):
    
    #if count > 4:
    #    break
    count += 1
    
    if idx > 290:
        train_query = row['sparql_query']


        msg = """
               Briefly explain the following query in logical steps as a chain of thought. 
               Explain in natural language. 
               Forget what you have about the query before. 
               Assume you are trying to construct the query again. 
               No comments. Output the steps only. 
               Do not include the original query in the explanation. 

               QUERY:{}
               THOUGHT:
        """
        msg = msg.format(train_query)

        response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "you are a helpful assistant focusing on DBpedia. \
                 You will explain SPARQL query in logical steps to help reconstruct the query. "},
                {"role": "user", "content": msg}
                ]
        )


        train_cot.append(response['choices'][0]['message']['content'].\
                                       strip().replace('\n', ' ' ))

100%|██████████| 408/408 [12:55<00:00,  1.90s/it]


In [ ]:
train['train_cot']=train_cot

In [770]:
#train.to_csv('../data/QALD/9/data/qald-9-train-with-embeddings-cot.csv', index=None)